In [1]:
import Auto_paint_self

In [11]:
CHM_PRE,lon,lat,levels,latlow,lattop,lonleft,lonright,times=Auto_paint_self.open_data_nc('one',r'E:\Multi-Sources_Precipitation_NC_2001_2022\CHM_PRE_China_2001_2022_2.nc','Pre','yes','time','2001-01-01','2022-12-31','yes','lon','yes','lat',18.0,53.5,73.5,135.0,0.25,0.25,'no','no',None,None,changeresolution=1,timespace=1,ifchange_west_east='no',ifinterpolate='no')
CMO_RPH,lon,lat,levels,latlow,lattop,lonleft,lonright,times=Auto_paint_self.open_data_nc('one',r'E:\Multi-Sources_Precipitation_NC_2001_2022\CMORPH_China_2001_2022_2.nc','Pre','yes','time','2001-01-01','2022-12-31','yes','lon','yes','lat',18.0,53.5,73.5,135.0,0.25,0.25,'no','no',None,None,changeresolution=1,timespace=1,ifchange_west_east='no',ifinterpolate='no')
ERA5_PRE,lon,lat,levels,latlow,lattop,lonleft,lonright,times=Auto_paint_self.open_data_nc('one',r'E:\Multi-Sources_Precipitation_NC_2001_2022\ERA5_China_2001_2022_2.nc','Pre','yes','time','2001-01-01','2022-12-31','yes','lon','yes','lat',18.0,53.5,73.5,135.0,0.25,0.25,'no','no',None,None,changeresolution=1,timespace=1,ifchange_west_east='no',ifinterpolate='no')
GPM_PRE,lon,lat,levels,latlow,lattop,lonleft,lonright,times=Auto_paint_self.open_data_nc('one',r'E:\Multi-Sources_Precipitation_NC_2001_2022\GPM_China_2001_2022_2.nc','Pre','yes','time','2001-01-01','2022-12-31','yes','lon','yes','lat',18.0,53.5,73.5,135.0,0.25,0.25,'no','no',None,None,changeresolution=1,timespace=1,ifchange_west_east='no',ifinterpolate='no')
GSMAP_PRE,lon,lat,levels,latlow,lattop,lonleft,lonright,times=Auto_paint_self.open_data_nc('one',r'E:\Multi-Sources_Precipitation_NC_2001_2022\GSMAP_China_2001_2022_2.nc','Pre','yes','time','2001-01-01','2022-12-31','yes','lon','yes','lat',18.0,53.5,73.5,135.0,0.25,0.25,'no','no',None,None,changeresolution=1,timespace=1,ifchange_west_east='no',ifinterpolate='no')
MSWEP_PRE,lon,lat,levels,latlow,lattop,lonleft,lonright,times=Auto_paint_self.open_data_nc('one',r'E:\Multi-Sources_Precipitation_NC_2001_2022\MSWEP_China_2001_2022_2.nc','Pre','yes','time','2001-01-01','2022-12-31','yes','lon','yes','lat',18.0,53.5,73.5,135.0,0.25,0.25,'no','no',None,None,changeresolution=1,timespace=1,ifchange_west_east='no',ifinterpolate='no')
PERSIANN_CDR_PRE,lon,lat,levels,latlow,lattop,lonleft,lonright,times=Auto_paint_self.open_data_nc('one',r'E:\Multi-Sources_Precipitation_NC_2001_2022\PERSIANN_CDR_PRE_China_2001_2022_2.nc','Pre','yes','time','2001-01-01','2022-12-31','yes','lon','yes','lat',18.0,53.5,73.5,135.0,0.25,0.25,'no','no',None,None,changeresolution=1,timespace=1,ifchange_west_east='no',ifinterpolate='no')

In [3]:
import numpy as np
vx=np.zeros((CMO_RPH.shape[0],CMO_RPH.shape[1],CMO_RPH.shape[2],6))
vy=np.zeros((CHM_PRE.shape[0],CHM_PRE.shape[1],CHM_PRE.shape[2]),dtype='int32')
CHM_PRE=np.nan_to_num(CHM_PRE,nan=0)
vx[:,:,:,0]=CMO_RPH
vx[:,:,:,1]=ERA5_PRE
vx[:,:,:,2]=GPM_PRE
vx[:,:,:,3]=GSMAP_PRE
vx[:,:,:,4]=MSWEP_PRE
vx[:,:,:,5]=PERSIANN_CDR_PRE
vx=np.nan_to_num(vx,nan=0)
for i in range(CHM_PRE.shape[1]):
    for j in range(CHM_PRE.shape[2]):
        vy[np.array(CHM_PRE[:,i,j])<0.1,i,j]=0
        vy[np.array(CHM_PRE[:,i,j])>=0.1,i,j]=1
vx=(vx-np.nanmean(vx,axis=0))/np.nanstd(vx,axis=0)
CHM_PRE=(CHM_PRE-np.nanmean(CHM_PRE,axis=0))/np.nanstd(CHM_PRE,axis=0)
vx=np.nan_to_num(vx,nan=0)
CHM_PRE=np.nan_to_num(CHM_PRE,nan=0)

C:\Users\TBYC\AppData\Local\Temp\ipykernel_75044\980890856.py:16: RuntimeWarning: invalid value encountered in divide
  vx=(vx-np.nanmean(vx,axis=0))/np.nanstd(vx,axis=0)
C:\Users\TBYC\AppData\Local\Temp\ipykernel_75044\980890856.py:17: RuntimeWarning: invalid value encountered in divide
  CHM_PRE=(CHM_PRE-np.nanmean(CHM_PRE,axis=0))/np.nanstd(CHM_PRE,axis=0)


In [4]:
print(np.isnan(vx).any(),np.isnan(CHM_PRE).any(),np.isnan(vy).any())

False False False


In [5]:
from keras.models import load_model
model=load_model(r'E:/xuanxuan/xuanxuan_unet_classify')
vx_last=model.predict(vx,batch_size=40)
vx_last[vx_last<=0.5]=0
vx_last[vx_last>0.5]=1
vx_new=np.zeros((CMO_RPH.shape[0],CMO_RPH.shape[1],CMO_RPH.shape[2],7))
vx_new[:,:,:,:6]=vx
vx_new[:,:,:,6]=vx_last.reshape(vx_last.shape[0],vx_last.shape[1],vx_last.shape[2])

201/201 [==============================] - 8s 24ms/step


In [6]:
print(np.isnan(vx_new).any())

False


In [7]:
from keras.models import load_model
index=int((1-0.2)*CHM_PRE.shape[0])
trainy=CHM_PRE[:index,:,:]
testy=CHM_PRE[index:,:,:]
trainx=vx_new[:index,:,:,:]
testx=vx_new[index:,:,:,:]
model=load_model(r'E:/xuanxuan/xuanxuan_unet_classify_regress')
predicty = model.predict(testx,batch_size=40)

41/41 [==============================] - 1s 26ms/step


In [8]:
time=times[index:]

In [9]:
predicty=predicty.reshape(predicty.shape[0],predicty.shape[1],predicty.shape[2])*np.nanstd(CHM_PRE,axis=0)+np.nanmean(CHM_PRE,axis=0)
testy=testy*np.nanstd(CHM_PRE,axis=0)+np.nanmean(CHM_PRE,axis=0)

In [10]:
Auto_paint_self.create_nc(time,lat,lon,None,predicty,'time','lat','lon','predicty','no',None,'yes','yes','yes','E:/xuanxuan/unet_pre_predicty.nc')
Auto_paint_self.create_nc(time,lat,lon,None,testy,'time','lat','lon','testy','no',None,'yes','yes','yes','E:/xuanxuan/unet_pre_testy.nc')

In [15]:
r=np.zeros((testx.shape[1],testx.shape[2]))
p=np.zeros((testx.shape[1],testx.shape[2]))
from scipy.stats import pearsonr
for i in range(testx.shape[1]):
    for j in range(testx.shape[2]):
        if np.isnan(CHM_PRE[:,i,j]).any():
            r[i,j],p[i,j]=np.nan,np.nan
        else:
            r[i,j],p[i,j] = pearsonr(predicty[:,i,j],testy[:,i,j])
print(np.nanmean(r))

0.6363264243490021


In [8]:
#用于生成unet网络
def Auto_Unet(vy,vx,downnum,covnum,test_size=0.25,task_mode='regression',if_best_mode='no',modelpath=None,ifrandom_split='yes',baselayer=64,cov_method='cov',cov_kernelsize=3,cov_strides=1,pool_method='maxpooling',pool_kernel_size=2,pool_strides=2,ifnormalization='no',normalization_method=None,if_weight_initialize='no',weight_initialize_method='TruncatedNormal',weight_initialize_parameter1=0.00,weight_initialize_parameter2=0.05,activate='tanh',if_last_act='no',if_print_model='yes',loss_function='default',optimizer='SGD',metrics='default',learning_rate=0.01,epochs=2000,batch_size=20,ifweight='yes',ifmute='no',ifsave='no',savepath=None,device='cpu'):
    import tensorflow as tf
    if device=='gpu':
        gpus = tf.config.list_physical_devices('GPU')
        if gpus:
            try:
                # 设置只使用 GPU 0
                tf.config.set_visible_devices(gpus[1], 'GPU')
                # 设置 GPU 0 的内存动态增长
                tf.config.experimental.set_memory_growth(gpus[1], True)
            except RuntimeError as e:
                print(e)
    from tensorflow.keras.models import Sequential,Model
    import math
    from tensorflow.python.keras.layers.core import Activation,Dropout,Dense,Flatten
    from tensorflow.keras.layers import BatchNormalization,LayerNormalization,LocallyConnected2D,Conv2D,MaxPooling2D,AveragePooling2D,Input,UpSampling2D,ZeroPadding2D
    from sklearn.model_selection import train_test_split
    import numpy as np
    import tensorflow.keras
    from tensorflow.keras.optimizers import SGD,Adam
    from scipy.stats import pearsonr
    from tensorflow.python.keras.layers.merge import concatenate
    import os
    from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score
    from keras.models import load_model
    import sklearn
    import copy
    import warnings

    warnings.filterwarnings("ignore", category=UserWarning, module='sklearn.metrics._classification')
    
    if task_mode=='regression':
        if loss_function=='default' or loss_function=='MeanSquaredError':
            loss=tf.keras.losses.MeanSquaredError()
        elif loss_function=='MeanSquaredError':
            loss=tf.keras.losses.MeanSquaredError()
        elif loss_function=='MeanAbsoluteError':
            loss=tf.keras.losses.MeanAbsoluteError()
        elif loss_function=='MeanAbsolutePercentageError':
            loss=tf.keras.losses.MeanAbsolutePercentageError()
        elif loss_function=='MeanSquaredLogarithmicError':
            loss=tf.keras.losses.MeanSquaredLogarithmicError()
        elif loss_function=='CosineSimilarity':
            loss=tf.keras.losses.CosineSimilarity()
        elif loss_function=='Huber':
            loss=tf.keras.losses.Huber()
        elif loss_function=='LogCosh':
            loss=tf.keras.losses.LogCosh()
        elif loss_function=='Pearsonr':
            def loss_pearsonr(y_true,y_pred):
                import tensorflow as tf
                y_true_mean=tf.reduce_mean(y_true,axis=0)
                y_pred_mean=tf.reduce_mean(y_pred,axis=0)
                cov=tf.reduce_sum((y_true-y_true_mean)*(y_pred-y_pred_mean),axis=0)
                y_true_v=tf.reduce_sum(tf.square((y_true-y_true_mean)),axis=0)
                y_pred_v=tf.reduce_sum(tf.square((y_pred-y_pred_mean)),axis=0)
                y_true_v=tf.sqrt(y_true_v)
                y_pred_v=tf.sqrt(y_pred_v)
                pearson=cov/(y_true_v*y_pred_v)
                return (1-pearson)**1.5
            loss=loss_pearsonr
        if metrics=='default' or metrics=='MeanSquaredError':
            metric=tf.keras.metrics.MeanSquaredError()
        elif metrics=='MeanAbsoluteError':
            metric=tf.keras.metrics.MeanAbsoluteError()
        elif metrics=='MeanAbsolutePercentageError':
            metric=tf.keras.metrics.MeanAbsolutePercentageError()
        elif metrics=='MeanSquaredLogarithmicError':
            metric=tf.keras.metrics.MeanSquaredLogarithmicError()
        elif metrics=='CosineSimilarity':
            metric=tf.keras.metrics.CosineSimilarity()
        elif metrics=='LogCoshError':
            metric=tf.keras.metrics.LogCoshError()
        elif metrics=='Pearsonr':
            def metrics_pearsonr(y_true,y_pred):
                import tensorflow as tf
                y_true_mean=tf.reduce_mean(y_true,axis=0)
                y_pred_mean=tf.reduce_mean(y_pred,axis=0)
                cov=tf.reduce_sum((y_true-y_true_mean)*(y_pred-y_pred_mean),axis=0)
                y_true_v=tf.reduce_sum(tf.square((y_true-y_true_mean)),axis=0)
                y_pred_v=tf.reduce_sum(tf.square((y_pred-y_pred_mean)),axis=0)
                y_true_v=tf.sqrt(y_true_v)
                y_pred_v=tf.sqrt(y_pred_v)
                pearson=cov/(y_true_v*y_pred_v)
                return (1-pearson)**1.5
            metric=metrics_pearsonr
    elif task_mode=='binary_classify':
        if loss_function=='default' or loss_function=='BinaryCrossentropy':
            loss=tf.keras.losses.BinaryCrossentropy()
        elif loss_function=='CategoricalCrossentropy':
            loss=tf.keras.losses.CategoricalCrossentropy()
        elif loss_function=='SparseCategoricalCrossentropy':
            loss=tf.keras.losses.SparseCategoricalCrossentropy()
        elif loss_function=='Poisson':
            loss=tf.keras.losses.Poisson()
        elif loss_function=='KLDivergence':
            loss=tf.keras.losses.KLDivergence()
        if metrics=='default' or metrics=='BinaryAccuracy':
            metric=tf.keras.metrics.BinaryAccuracy()
        elif metrics=='MeanAbsoluteError':
            metric=tf.keras.metrics.MeanAbsoluteError()
        elif metrics=='Accuracy':
            metric=tf.keras.metrics.Accuracy()
        elif metrics=='CategoricalAccuracy':
            metric=tf.keras.metrics.CategoricalAccuracy()
        elif metrics=='SparseCategoricalAccuracy':
            metric=tf.keras.metrics.SparseCategoricalAccuracy()
        elif metrics=='TopKCategoricalAccuracy':
            metric=tf.keras.metrics.TopKCategoricalAccuracy()
        elif metrics=='SparseTopKCategoricalAccuracy':
            metric=tf.keras.metrics.SparseTopKCategoricalAccuracy()
        elif metrics=='BinaryCrossentropy':
            metric=tf.keras.metrics.BinaryCrossentropy()
        elif metrics=='CategoricalCrossentropy':
            metric=tf.keras.metrics.CategoricalCrossentropy()
        elif metrics=='SparseCategoricalCrossentropy':
            metric=tf.keras.metrics.SparseCategoricalCrossentropy()
        elif metrics=='Accuracy':
            metric=tf.keras.metrics.Accuracy()
        elif metrics=='CategoricalAccuracy':
            metric=tf.keras.metrics.CategoricalAccuracy()
        elif metrics=='SparseCategoricalAccuracy':
            metric=tf.keras.metrics.SparseCategoricalAccuracy()
        elif metrics=='KLDivergence':
            metric=tf.keras.metrics.KLDivergence()
        elif metrics=='Poisson':
            metric=tf.keras.metrics.Poisson()
        elif metrics=='AUC':
            metric=tf.keras.metrics.AUC()
        elif metrics=='Precision':
            metric=tf.keras.metrics.Precision()
        elif metrics=='Recall':
            metric=tf.keras.metrics.Recall()
        elif metrics=='TruePositives':
            metric=tf.keras.metrics.TruePositives()
        elif metrics=='TrueNegatives':
            metric=tf.keras.metrics.TrueNegatives()
        elif metrics=='FalsePositives':
            metric=tf.keras.metrics.FalsePositives()
        elif metrics=='FalseNegatives':
            metric=tf.keras.metrics.FalseNegatives()
        elif metrics=='PrecisionAtRecall':
            metric=tf.keras.metrics.PrecisionAtRecall()
        elif metrics=='SensitivityAtSpecificity':
            metric=tf.keras.metrics.SensitivityAtSpecificity()
        elif metrics=='SpecificityAtSensitivity':
            metric=tf.keras.metrics.SpecificityAtSensitivity()
    elif task_mode=='multi_classify':
        if loss_function=='default' or loss_function=='CategoricalCrossentropy':
            loss=tf.keras.losses.CategoricalCrossentropy()
        elif loss_function=='SparseCategoricalCrossentropy':
            loss=tf.keras.losses.SparseCategoricalCrossentropy()
        elif loss_function=='Poisson':
            loss=tf.keras.losses.Poisson()
        elif loss_function=='KLDivergence':
            loss=tf.keras.losses.KLDivergence()
        if metrics=='default' or metrics=='Accuracy':
            metric=tf.keras.metrics.Accuracy()
        elif metrics=='MeanAbsoluteError':
            metric=tf.keras.metrics.MeanAbsoluteError()
        elif metrics=='CategoricalAccuracy':
            metric=tf.keras.metrics.CategoricalAccuracy()
        elif metrics=='SparseCategoricalAccuracy':
            metric=tf.keras.metrics.SparseCategoricalAccuracy()
        elif metrics=='TopKCategoricalAccuracy':
            metric=tf.keras.metrics.TopKCategoricalAccuracy()
        elif metrics=='SparseTopKCategoricalAccuracy':
            metric=tf.keras.metrics.SparseTopKCategoricalAccuracy()
        elif metrics=='CategoricalCrossentropy':
            metric=tf.keras.metrics.CategoricalCrossentropy()
        elif metrics=='SparseCategoricalCrossentropy':
            metric=tf.keras.metrics.SparseCategoricalCrossentropy()
        elif metrics=='Accuracy':
            metric=tf.keras.metrics.Accuracy()
        elif metrics=='CategoricalAccuracy':
            metric=tf.keras.metrics.CategoricalAccuracy()
        elif metrics=='SparseCategoricalAccuracy':
            metric=tf.keras.metrics.SparseCategoricalAccuracy()
        elif metrics=='KLDivergence':
            metric=tf.keras.metrics.KLDivergence()
        elif metrics=='Poisson':
            metric=tf.keras.metrics.Poisson()
    weights=0
    model=0
    if vy.ndim==3:
        vy=vy.reshape(vy.shape[0],vy.shape[1],vy.shape[2],1)
    if device=='cpu':
        os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
        with tf.device('/cpu:0'):
            model=None
            predicty=None
            testy=None
            r=None
            p=None
            k=0
            total_covnum=0
            if ifrandom_split=='yes':
                trainy,testy,trainx,testx = train_test_split(vy,vx,test_size=test_size,random_state=25)
            else:
                index=int((1-test_size)*vy.shape[0])
                trainy=vy[:index,:,:,:]
                testy=vy[index:,:,:,:]
                trainx=vx[:index,:,:,:]
                testx=vx[index:,:,:,:]
            if if_best_mode=='no':
                W=trainx.shape[1]
                H=trainx.shape[2]
                for i in range(int(downnum-1)):
                    W=math.floor(((W-pool_kernel_size)/pool_strides)+1)
                    H=math.floor(((H-pool_kernel_size)/pool_strides)+1)
                if W<1 or H<1:
                    print('下采样次数过多,导致最下层图形长度/宽度小于1!')
                    return model,predicty,testy,r,p
                if pool_strides!=pool_kernel_size:
                    print('池化步长和池化核不等,将导致下采样前后通道数相除不为整数！')
                    return model,predicty,testy,r,p
                upsize=pool_strides
                inputs=Input((trainx.shape[1],trainx.shape[2],trainx.shape[3]))
                for i in range(int(downnum)):
                    for j in range(int(covnum)):
                        if cov_method=='cov':
                            if if_weight_initialize=='no':
                                if total_covnum ==0:
                                    exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((trainx.shape[1]-(trainx.shape[1]/cov_strides)))/2),math.ceil(math.floor((trainx.shape[1]-(trainx.shape[1]/cov_strides)))/2)),(math.floor(math.floor((trainx.shape[2]-(trainx.shape[2]/cov_strides)))/2),math.ceil(math.floor((trainx.shape[2]-(trainx.shape[2]/cov_strides)))/2))))(Conv2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,padding="same")(inputs))')
                                else:
                                    if j==0:
                                        exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((pool'+str(i)+'.shape[1]-(pool'+str(i)+'.shape[1]/cov_strides)))/2),math.ceil(math.floor((pool'+str(i)+'.shape[1]-(pool'+str(i)+'.shape[1]/cov_strides)))/2)),(math.floor(math.floor((pool'+str(i)+'.shape[2]-(pool'+str(i)+'.shape[2]/cov_strides)))/2),math.ceil(math.floor((pool'+str(i)+'.shape[2]-(pool'+str(i)+'.shape[2]/cov_strides)))/2))))(Conv2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,padding="same")(pool'+str(i)+'))')
                                    else:
                                        if ifnormalization=='yes':
                                            exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((norm'+str(i+1)+'.shape[1]-(norm'+str(i+1)+'.shape[1]/cov_strides)))/2),math.ceil(math.floor((norm'+str(i+1)+'.shape[1]-(norm'+str(i+1)+'.shape[1]/cov_strides)))/2)),(math.floor(math.floor((norm'+str(i+1)+'.shape[2]-(norm'+str(i+1)+'.shape[2]/cov_strides)))/2),math.ceil(math.floor((norm'+str(i+1)+'.shape[2]-(norm'+str(i+1)+'.shape[2]/cov_strides)))/2))))(Conv2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,padding="same")(norm'+str(i+1)+'))')
                                        else:
                                            exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((cov'+str(i+1)+'.shape[1]-(cov'+str(i+1)+'.shape[1]/cov_strides)))/2),math.ceil(math.floor((cov'+str(i+1)+'.shape[1]-(cov'+str(i+1)+'.shape[1]/cov_strides)))/2)),(math.floor(math.floor((cov'+str(i+1)+'.shape[2]-(cov'+str(i+1)+'.shape[2]/cov_strides)))/2),math.ceil(math.floor((cov'+str(i+1)+'.shape[2]-(cov'+str(i+1)+'.shape[2]/cov_strides)))/2))))(Conv2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,padding="same")(cov'+str(i+1)+'))')
                            elif if_weight_initialize=='yes':
                                if total_covnum==0:
                                    if weight_initialize_method=='RandomNormal':
                                        exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((trainx.shape[1]-(trainx.shape[1]/cov_strides)))/2),math.ceil(math.floor((trainx.shape[1]-(trainx.shape[1]/cov_strides)))/2)),(math.floor(math.floor((trainx.shape[2]-(trainx.shape[2]/cov_strides)))/2),math.ceil(math.floor((trainx.shape[2]-(trainx.shape[2]/cov_strides)))/2))))(Conv2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,padding="same",kernel_initializer = RandomNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2))(inputs))')
                                    elif weight_initialize_method=='RandomUniform':
                                        exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((trainx.shape[1]-(trainx.shape[1]/cov_strides)))/2),math.ceil(math.floor((trainx.shape[1]-(trainx.shape[1]/cov_strides)))/2)),(math.floor(math.floor((trainx.shape[2]-(trainx.shape[2]/cov_strides)))/2),math.ceil(math.floor((trainx.shape[2]-(trainx.shape[2]/cov_strides)))/2))))(Conv2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,padding="same",kernel_initializer = RandomUniform(minval=weight_initialize_parameter1,maxval=weight_initialize_parameter2))(inputs))')
                                    elif weight_initialize_method=='TruncatedNormal':
                                        exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((trainx.shape[1]-(trainx.shape[1]/cov_strides)))/2),math.ceil(math.floor((trainx.shape[1]-(trainx.shape[1]/cov_strides)))/2)),(math.floor(math.floor((trainx.shape[2]-(trainx.shape[2]/cov_strides)))/2),math.ceil(math.floor((trainx.shape[2]-(trainx.shape[2]/cov_strides)))/2))))(Conv2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,padding="same", kernel_initializer = TruncatedNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2,metrics="MeanSquaredError"))(inputs))')
                                else:
                                    if j==0:
                                        if weight_initialize_method=='RandomNormal':
                                            exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((pool'+str(i)+'.shape[1]-(pool'+str(i)+'.shape[1]/cov_strides)))/2),math.ceil(math.floor((pool'+str(i)+'.shape[1]-(pool'+str(i)+'.shape[1]/cov_strides)))/2)),(math.floor(math.floor((pool'+str(i)+'.shape[2]-(pool'+str(i)+'.shape[2]/cov_strides)))/2),math.ceil(math.floor((pool'+str(i)+'.shape[2]-(pool'+str(i)+'.shape[2]/cov_strides)))/2))))(Conv2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,padding="same",kernel_initializer = RandomNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2))(pool'+str(i)+'))')
                                        elif weight_initialize_method=='RandomUniform':
                                            exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((pool'+str(i)+'.shape[1]-(pool'+str(i)+'.shape[1]/cov_strides)))/2),math.ceil(math.floor((pool'+str(i)+'.shape[1]-(pool'+str(i)+'.shape[1]/cov_strides)))/2)),(math.floor(math.floor((pool'+str(i)+'.shape[2]-(pool'+str(i)+'.shape[2]/cov_strides)))/2),math.ceil(math.floor((pool'+str(i)+'.shape[2]-(pool'+str(i)+'.shape[2]/cov_strides)))/2))))(Conv2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,padding="same",kernel_initializer = RandomUniform(minval=weight_initialize_parameter1,maxval=weight_initialize_parameter2))(pool'+str(i)+'))')
                                        elif weight_initialize_method=='TruncatedNormal':
                                            exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((pool'+str(i)+'.shape[1]-(pool'+str(i)+'.shape[1]/cov_strides)))/2),math.ceil(math.floor((pool'+str(i)+'.shape[1]-(pool'+str(i)+'.shape[1]/cov_strides)))/2)),(math.floor(math.floor((pool'+str(i)+'.shape[2]-(pool'+str(i)+'.shape[2]/cov_strides)))/2),math.ceil(math.floor((pool'+str(i)+'.shape[2]-(pool'+str(i)+'.shape[2]/cov_strides)))/2))))(Conv2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,padding="same",kernel_initializer = TruncatedNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2,metrics="MeanSquaredError"))(pool'+str(i)+'))')
                                    else:
                                        if ifnormalization=='yes':
                                            if weight_initialize_method=='RandomNormal':
                                                exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((norm'+str(i+1)+'.shape[1]-(norm'+str(i+1)+'.shape[1]/cov_strides)))/2),math.ceil(math.floor((norm'+str(i+1)+'.shape[1]-(norm'+str(i+1)+'.shape[1]/cov_strides)))/2)),(math.floor(math.floor((norm'+str(i+1)+'.shape[2]-(norm'+str(i+1)+'.shape[2]/cov_strides)))/2),math.ceil(math.floor((norm'+str(i+1)+'.shape[2]-(norm'+str(i+1)+'.shape[2]/cov_strides)))/2))))(Conv2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,padding="same",kernel_initializer = RandomNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2))(norm'+str(i+1)+'))')
                                            elif weight_initialize_method=='RandomUniform':
                                                exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((norm'+str(i+1)+'.shape[1]-(norm'+str(i+1)+'.shape[1]/cov_strides)))/2),math.ceil(math.floor((norm'+str(i+1)+'.shape[1]-(norm'+str(i+1)+'.shape[1]/cov_strides)))/2)),(math.floor(math.floor((norm'+str(i+1)+'.shape[2]-(norm'+str(i+1)+'.shape[2]/cov_strides)))/2),math.ceil(math.floor((norm'+str(i+1)+'.shape[2]-(norm'+str(i+1)+'.shape[2]/cov_strides)))/2))))(Conv2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,padding="same",kernel_initializer = RandomUniform(minval=weight_initialize_parameter1,maxval=weight_initialize_parameter2))(norm'+str(i+1)+'))')
                                            elif weight_initialize_method=='TruncatedNormal':
                                                exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((norm'+str(i+1)+'.shape[1]-(norm'+str(i+1)+'.shape[1]/cov_strides)))/2),math.ceil(math.floor((norm'+str(i+1)+'.shape[1]-(norm'+str(i+1)+'.shape[1]/cov_strides)))/2)),(math.floor(math.floor((norm'+str(i+1)+'.shape[2]-(norm'+str(i+1)+'.shape[2]/cov_strides)))/2),math.ceil(math.floor((norm'+str(i+1)+'.shape[2]-(norm'+str(i+1)+'.shape[2]/cov_strides)))/2))))(Conv2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,padding="same",kernel_initializer = TruncatedNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2,metrics="MeanSquaredError"))(norm'+str(i+1)+'))')
                                        else:
                                            if weight_initialize_method=='RandomNormal':
                                                exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((cov'+str(i+1)+'.shape[1]-(cov'+str(i+1)+'.shape[1]/cov_strides)))/2),math.ceil(math.floor((cov'+str(i+1)+'.shape[1]-(cov'+str(i+1)+'.shape[1]/cov_strides)))/2)),(math.floor(math.floor((cov'+str(i+1)+'.shape[2]-(cov'+str(i+1)+'.shape[2]/cov_strides)))/2),math.ceil(math.floor((cov'+str(i+1)+'.shape[2]-(cov'+str(i+1)+'.shape[2]/cov_strides)))/2))))(Conv2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,padding="same",kernel_initializer = RandomNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2))(cov'+str(i+1)+'))')
                                            elif weight_initialize_method=='RandomUniform':
                                                exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((cov'+str(i+1)+'.shape[1]-(cov'+str(i+1)+'.shape[1]/cov_strides)))/2),math.ceil(math.floor((cov'+str(i+1)+'.shape[1]-(cov'+str(i+1)+'.shape[1]/cov_strides)))/2)),(math.floor(math.floor((cov'+str(i+1)+'.shape[2]-(cov'+str(i+1)+'.shape[2]/cov_strides)))/2),math.ceil(math.floor((cov'+str(i+1)+'.shape[2]-(cov'+str(i+1)+'.shape[2]/cov_strides)))/2))))(Conv2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,padding="same",kernel_initializer = RandomUniform(minval=weight_initialize_parameter1,maxval=weight_initialize_parameter2))(cov'+str(i+1)+'))')
                                            elif weight_initialize_method=='TruncatedNormal':
                                                exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((cov'+str(i+1)+'.shape[1]-(cov'+str(i+1)+'.shape[1]/cov_strides)))/2),math.ceil(math.floor((cov'+str(i+1)+'.shape[1]-(cov'+str(i+1)+'.shape[1]/cov_strides)))/2)),(math.floor(math.floor((cov'+str(i+1)+'.shape[2]-(cov'+str(i+1)+'.shape[2]/cov_strides)))/2),math.ceil(math.floor((cov'+str(i+1)+'.shape[2]-(cov'+str(i+1)+'.shape[2]/cov_strides)))/2))))(Conv2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,padding="same",kernel_initializer = TruncatedNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2,metrics="MeanSquaredError"))(cov'+str(i+1)+'))')
                        elif cov_method=='localcov':
                            if if_weight_initialize=='no':
                                if total_covnum==0:
                                    exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((trainx.shape[1]-((trainx.shape[1]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((trainx.shape[1]-((trainx.shape[1]-cov_kernelsize+1)/cov_strides)))/2)),(math.floor(math.floor((trainx.shape[2]-((trainx.shape[2]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((trainx.shape[2]-((trainx.shape[2]-cov_kernelsize+1)/cov_strides)))/2))))(LocallyConnected2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate)(inputs))')
                                else:
                                    if j==0:
                                        exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((pool'+str(i)+'.shape[1]-((pool'+str(i)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((pool'+str(i)+'.shape[1]-((pool'+str(i)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2)),(math.floor(math.floor((pool'+str(i)+'.shape[2]-((pool'+str(i)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((pool'+str(i)+'.shape[2]-((pool'+str(i)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2))))(LocallyConnected2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate)(pool'+str(i)+'))')
                                    else:
                                        if ifnormalization=='yes':
                                            exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((norm'+str(i+1)+'.shape[1]-((norm'+str(i+1)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((norm'+str(i+1)+'.shape[1]-((norm'+str(i+1)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2)),(math.floor(math.floor((norm'+str(i+1)+'.shape[2]-((norm'+str(i+1)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((norm'+str(i+1)+'.shape[2]-((norm'+str(i+1)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2))))(LocallyConnected2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate)(norm'+str(i+1)+'))')
                                        else:
                                            exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((cov'+str(i+1)+'.shape[1]-((cov'+str(i+1)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((cov'+str(i+1)+'.shape[1]-((cov'+str(i+1)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2)),(math.floor(math.floor((cov'+str(i+1)+'.shape[2]-((cov'+str(i+1)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((cov'+str(i+1)+'.shape[2]-((cov'+str(i+1)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2))))(LocallyConnected2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate)(cov'+str(i+1)+'))')
                            elif if_weight_initialize=='yes':
                                if total_covnum==0:
                                    if weight_initialize_method=='RandomNormal':
                                        exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((trainx.shape[1]-((trainx.shape[1]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((trainx.shape[1]-((trainx.shape[1]-cov_kernelsize+1)/cov_strides)))/2)),(math.floor(math.floor((trainx.shape[2]-((trainx.shape[2]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((trainx.shape[2]-((trainx.shape[2]-cov_kernelsize+1)/cov_strides)))/2))))(LocallyConnected2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate, kernel_initializer = RandomNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2))(inputs))')
                                    elif weight_initialize_method=='RandomUniform':
                                        exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((trainx.shape[1]-((trainx.shape[1]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((trainx.shape[1]-((trainx.shape[1]-cov_kernelsize+1)/cov_strides)))/2)),(math.floor(math.floor((trainx.shape[2]-((trainx.shape[2]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((trainx.shape[2]-((trainx.shape[2]-cov_kernelsize+1)/cov_strides)))/2))))(LocallyConnected2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate, kernel_initializer = RandomUniform(minval=weight_initialize_parameter1,maxval=weight_initialize_parameter2))(inputs))')
                                    elif weight_initialize_method=='TruncatedNormal':
                                        exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((trainx.shape[1]-((trainx.shape[1]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((trainx.shape[1]-((trainx.shape[1]-cov_kernelsize+1)/cov_strides)))/2)),(math.floor(math.floor((trainx.shape[2]-((trainx.shape[2]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((trainx.shape[2]-((trainx.shape[2]-cov_kernelsize+1)/cov_strides)))/2))))(LocallyConnected2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate, kernel_initializer = TruncatedNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2,metrics="MeanSquaredError"))(inputs))')
                                else:
                                    if j==0:
                                        if weight_initialize_method=='RandomNormal':
                                            exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((pool'+str(i)+'.shape[1]-((pool'+str(i)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((pool'+str(i)+'.shape[1]-((pool'+str(i)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2)),(math.floor(math.floor((pool'+str(i)+'.shape[2]-((pool'+str(i)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((pool'+str(i)+'.shape[2]-((pool'+str(i)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2))))(LocallyConnected2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,kernel_initializer = RandomNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2))(pool'+str(i)+'))')
                                        elif weight_initialize_method=='RandomUniform':
                                            exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((pool'+str(i)+'.shape[1]-((pool'+str(i)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((pool'+str(i)+'.shape[1]-((pool'+str(i)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2)),(math.floor(math.floor((pool'+str(i)+'.shape[2]-((pool'+str(i)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((pool'+str(i)+'.shape[2]-((pool'+str(i)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2))))(LocallyConnected2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,kernel_initializer = RandomUniform(minval=weight_initialize_parameter1,maxval=weight_initialize_parameter2))(pool'+str(i)+'))')
                                        elif weight_initialize_method=='TruncatedNormal':
                                            exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((pool'+str(i)+'.shape[1]-((pool'+str(i)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((pool'+str(i)+'.shape[1]-((pool'+str(i)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2)),(math.floor(math.floor((pool'+str(i)+'.shape[2]-((pool'+str(i)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((pool'+str(i)+'.shape[2]-((pool'+str(i)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2))))(LocallyConnected2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,kernel_initializer = TruncatedNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2,metrics="MeanSquaredError"))(pool'+str(i)+'))')
                                    else:
                                        if ifnormalization=='yes':
                                            if weight_initialize_method=='RandomNormal':
                                                exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((norm'+str(i+1)+'.shape[1]-((norm'+str(i+1)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((norm'+str(i+1)+'.shape[1]-((norm'+str(i+1)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2)),(math.floor(math.floor((norm'+str(i+1)+'.shape[2]-((norm'+str(i+1)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((norm'+str(i+1)+'.shape[2]-((norm'+str(i+1)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2))))(LocallyConnected2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,kernel_initializer = RandomNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2))(norm'+str(i+1)+'))')
                                            elif weight_initialize_method=='RandomUniform':
                                                exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((norm'+str(i+1)+'.shape[1]-((norm'+str(i+1)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((norm'+str(i+1)+'.shape[1]-((norm'+str(i+1)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2)),(math.floor(math.floor((norm'+str(i+1)+'.shape[2]-((norm'+str(i+1)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((norm'+str(i+1)+'.shape[2]-((norm'+str(i+1)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2))))(LocallyConnected2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,kernel_initializer = RandomUniform(minval=weight_initialize_parameter1,maxval=weight_initialize_parameter2))(norm'+str(i+1)+'))')
                                            elif weight_initialize_method=='TruncatedNormal':
                                                exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((norm'+str(i+1)+'.shape[1]-((norm'+str(i+1)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((norm'+str(i+1)+'.shape[1]-((norm'+str(i+1)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2)),(math.floor(math.floor((norm'+str(i+1)+'.shape[2]-((norm'+str(i+1)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((norm'+str(i+1)+'.shape[2]-((norm'+str(i+1)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2))))(LocallyConnected2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,kernel_initializer = TruncatedNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2,metrics="MeanSquaredError"))(norm'+str(i+1)+'))')
                                        else:
                                            if weight_initialize_method=='RandomNormal':
                                                exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((cov'+str(i+1)+'.shape[1]-((cov'+str(i+1)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((cov'+str(i+1)+'.shape[1]-((cov'+str(i+1)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2)),(math.floor(math.floor((cov'+str(i+1)+'.shape[2]-((cov'+str(i+1)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((cov'+str(i+1)+'.shape[2]-((cov'+str(i+1)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2))))(LocallyConnected2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,kernel_initializer = RandomNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2))(cov'+str(i+1)+'))')
                                            elif weight_initialize_method=='RandomUniform':
                                                exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((cov'+str(i+1)+'.shape[1]-((cov'+str(i+1)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((cov'+str(i+1)+'.shape[1]-((cov'+str(i+1)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2)),(math.floor(math.floor((cov'+str(i+1)+'.shape[2]-((cov'+str(i+1)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((cov'+str(i+1)+'.shape[2]-((cov'+str(i+1)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2))))(LocallyConnected2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,kernel_initializer = RandomUniform(minval=weight_initialize_parameter1,maxval=weight_initialize_parameter2))(cov'+str(i+1)+'))')
                                            elif weight_initialize_method=='TruncatedNormal':
                                                exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((cov'+str(i+1)+'.shape[1]-((cov'+str(i+1)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((cov'+str(i+1)+'.shape[1]-((cov'+str(i+1)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2)),(math.floor(math.floor((cov'+str(i+1)+'.shape[2]-((cov'+str(i+1)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((cov'+str(i+1)+'.shape[2]-((cov'+str(i+1)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2))))(LocallyConnected2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,kernel_initializer = TruncatedNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2,metrics="MeanSquaredError"))(cov'+str(i+1)+'))')
                        total_covnum=total_covnum+1
                        if ifnormalization=='yes':
                            if normalization_method=='batchnormalization':
                                exec('norm'+str(i+1)+'=BatchNormalization(axis=-1)(cov'+str(i+1)+')')
                            elif normalization_method=='layernormalization':
                                exec('norm'+str(i+1)+'=LayerNormalization(axis=-1)(cov'+str(i+1)+')')
                    k=k+1
                    if i!=(downnum-1):
                        baselayer=baselayer*2
                    if pool_method=='maxpooling' and i!=(downnum-1):
                        if ifnormalization=='yes':
                            exec('pool'+str(i+1)+'=MaxPooling2D(pool_size=(pool_kernel_size,pool_kernel_size),strides=pool_strides)(norm'+str(i+1)+')')
                        else:
                            exec('pool'+str(i+1)+'=MaxPooling2D(pool_size=(pool_kernel_size,pool_kernel_size),strides=pool_strides)(cov'+str(i+1)+')')
                    elif pool_method=='avepooling' and i!=(downnum-1):
                        if ifnormalization=='yes':
                            exec('pool'+str(i+1)+'=AveragePooling2D(pool_size=(pool_kernel_size,pool_kernel_size),strides=pool_strides)(norm'+str(i+1)+')')
                        else:
                            exec('pool'+str(i+1)+'=AveragePooling2D(pool_size=(pool_kernel_size,pool_kernel_size),strides=pool_strides)(cov'+str(i+1)+')')
                l=k-1
                k=k+1
                for i in range(downnum,2*downnum-1):
                    baselayer=int(baselayer/2)
                    if cov_method=='cov':
                        if if_weight_initialize=='no':
                            if ifnormalization=='yes':
                                exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((norm'+str(i)+'.shape[1]-(norm'+str(i)+'.shape[1]/cov_strides)))/2),math.ceil(math.floor((norm'+str(i)+'.shape[1]-(norm'+str(i)+'.shape[1]/cov_strides)))/2)),(math.floor(math.floor((norm'+str(i)+'.shape[2]-(norm'+str(i)+'.shape[2]/cov_strides)))/2),math.ceil(math.floor((norm'+str(i)+'.shape[2]-(norm'+str(i)+'.shape[2]/cov_strides)))/2))))(Conv2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,padding="same")(norm'+str(i)+'))')
                            else:
                                exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((cov'+str(i)+'.shape[1]-(cov'+str(i)+'.shape[1]/cov_strides)))/2),math.ceil(math.floor((cov'+str(i)+'.shape[1]-(cov'+str(i)+'.shape[1]/cov_strides)))/2)),(math.floor(math.floor((cov'+str(i)+'.shape[2]-(cov'+str(i)+'.shape[2]/cov_strides)))/2),math.ceil(math.floor((cov'+str(i)+'.shape[2]-(cov'+str(i)+'.shape[2]/cov_strides)))/2))))(Conv2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,padding="same")(cov'+str(i)+'))')
                        elif if_weight_initialize=='yes':
                            if ifnormalization=='yes':
                                if weight_initialize_method=='RandomNormal':
                                    exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((norm'+str(i)+'.shape[1]-(norm'+str(i)+'.shape[1]/cov_strides)))/2),math.ceil(math.floor((norm'+str(i)+'.shape[1]-(norm'+str(i)+'.shape[1]/cov_strides)))/2)),(math.floor(math.floor((norm'+str(i)+'.shape[2]-(norm'+str(i)+'.shape[2]/cov_strides)))/2),math.ceil(math.floor((norm'+str(i)+'.shape[2]-(norm'+str(i)+'.shape[2]/cov_strides)))/2))))(Conv2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,padding="same",kernel_initializer = RandomNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2))(norm'+str(i)+'))')
                                elif weight_initialize_method=='RandomUniform':
                                    exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((norm'+str(i)+'.shape[1]-(norm'+str(i)+'.shape[1]/cov_strides)))/2),math.ceil(math.floor((norm'+str(i)+'.shape[1]-(norm'+str(i)+'.shape[1]/cov_strides)))/2)),(math.floor(math.floor((norm'+str(i)+'.shape[2]-(norm'+str(i)+'.shape[2]/cov_strides)))/2),math.ceil(math.floor((norm'+str(i)+'.shape[2]-(norm'+str(i)+'.shape[2]/cov_strides)))/2))))(Conv2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,padding="same",kernel_initializer = RandomUniform(minval=weight_initialize_parameter1,maxval=weight_initialize_parameter2))(norm'+str(i)+'))')
                                elif weight_initialize_method=='TruncatedNormal':
                                    exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((norm'+str(i)+'.shape[1]-(norm'+str(i)+'.shape[1]/cov_strides)))/2),math.ceil(math.floor((norm'+str(i)+'.shape[1]-(norm'+str(i)+'.shape[1]/cov_strides)))/2)),(math.floor(math.floor((norm'+str(i)+'.shape[2]-(norm'+str(i)+'.shape[2]/cov_strides)))/2),math.ceil(math.floor((norm'+str(i)+'.shape[2]-(norm'+str(i)+'.shape[2]/cov_strides)))/2))))(Conv2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,padding="same",kernel_initializer = TruncatedNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2,metrics="MeanSquaredError"))(norm'+str(i)+'))')
                            else:
                                if weight_initialize_method=='RandomNormal':
                                    exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((cov'+str(i)+'.shape[1]-(cov'+str(i)+'.shape[1]/cov_strides)))/2),math.ceil(math.floor((cov'+str(i)+'.shape[1]-(cov'+str(i)+'.shape[1]/cov_strides)))/2)),(math.floor(math.floor((cov'+str(i)+'.shape[2]-(cov'+str(i)+'.shape[2]/cov_strides)))/2),math.ceil(math.floor((cov'+str(i)+'.shape[2]-(cov'+str(i)+'.shape[2]/cov_strides)))/2))))(Conv2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,padding="same",kernel_initializer = RandomNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2))(cov'+str(i)+'))')
                                elif weight_initialize_method=='RandomUniform':
                                    exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((cov'+str(i)+'.shape[1]-(cov'+str(i)+'.shape[1]/cov_strides)))/2),math.ceil(math.floor((cov'+str(i)+'.shape[1]-(cov'+str(i)+'.shape[1]/cov_strides)))/2)),(math.floor(math.floor((cov'+str(i)+'.shape[2]-(cov'+str(i)+'.shape[2]/cov_strides)))/2),math.ceil(math.floor((cov'+str(i)+'.shape[2]-(cov'+str(i)+'.shape[2]/cov_strides)))/2))))(Conv2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,padding="same",kernel_initializer = RandomUniform(minval=weight_initialize_parameter1,maxval=weight_initialize_parameter2))(cov'+str(i)+'))')
                                elif weight_initialize_method=='TruncatedNormal':
                                    exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((cov'+str(i)+'.shape[1]-(cov'+str(i)+'.shape[1]/cov_strides)))/2),math.ceil(math.floor((cov'+str(i)+'.shape[1]-(cov'+str(i)+'.shape[1]/cov_strides)))/2)),(math.floor(math.floor((cov'+str(i)+'.shape[2]-(cov'+str(i)+'.shape[2]/cov_strides)))/2),math.ceil(math.floor((cov'+str(i)+'.shape[2]-(cov'+str(i)+'.shape[2]/cov_strides)))/2))))(Conv2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,padding="same",kernel_initializer = TruncatedNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2,metrics="MeanSquaredError"))(cov'+str(i)+'))')
                    elif cov_method=='localcov':
                        if if_weight_initialize=='no':
                            if ifnormalization=='yes':
                                exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((norm'+str(i)+'.shape[1]-((norm'+str(i)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((norm'+str(i)+'.shape[1]-((norm'+str(i)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2)),(math.floor(math.floor((norm'+str(i)+'.shape[2]-((norm'+str(i)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((norm'+str(i)+'.shape[2]-((norm'+str(i)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2))))(LocallyConnected2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate)(norm'+str(i)+'))')
                            else:
                                exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((cov'+str(i)+'.shape[1]-((cov'+str(i)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((cov'+str(i)+'.shape[1]-((cov'+str(i)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2)),(math.floor(math.floor((cov'+str(i)+'.shape[2]-((cov'+str(i)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((cov'+str(i)+'.shape[2]-((cov'+str(i)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2))))(LocallyConnected2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate)(cov'+str(i)+'))')
                        elif if_weight_initialize=='yes':
                            if ifnormalization=='yes':
                                if weight_initialize_method=='RandomNormal':
                                    exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((norm'+str(i)+'.shape[1]-((norm'+str(i)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((norm'+str(i)+'.shape[1]-((norm'+str(i)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2)),(math.floor(math.floor((norm'+str(i)+'.shape[2]-((norm'+str(i)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((norm'+str(i)+'.shape[2]-((norm'+str(i)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2))))(LocallyConnected2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,kernel_initializer = RandomNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2))(norm'+str(i)+'))')
                                elif weight_initialize_method=='RandomUniform':
                                    exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((norm'+str(i)+'.shape[1]-((norm'+str(i)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((norm'+str(i)+'.shape[1]-((norm'+str(i)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2)),(math.floor(math.floor((norm'+str(i)+'.shape[2]-((norm'+str(i)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((norm'+str(i)+'.shape[2]-((norm'+str(i)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2))))(LocallyConnected2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,kernel_initializer = RandomUniform(minval=weight_initialize_parameter1,maxval=weight_initialize_parameter2))(norm'+str(i)+'))')
                                elif weight_initialize_method=='TruncatedNormal':
                                    exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((norm'+str(i)+'.shape[1]-((norm'+str(i)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((norm'+str(i)+'.shape[1]-((norm'+str(i)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2)),(math.floor(math.floor((norm'+str(i)+'.shape[2]-((norm'+str(i)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((norm'+str(i)+'.shape[2]-((norm'+str(i)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2))))(LocallyConnected2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,kernel_initializer = TruncatedNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2,metrics="MeanSquaredError"))(norm'+str(i)+'))')
                            else:
                                if weight_initialize_method=='RandomNormal':
                                    exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((cov'+str(i)+'.shape[1]-((cov'+str(i)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((cov'+str(i)+'.shape[1]-((cov'+str(i)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2)),(math.floor(math.floor((cov'+str(i)+'.shape[2]-((cov'+str(i)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((cov'+str(i)+'.shape[2]-((cov'+str(i)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2))))(LocallyConnected2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,kernel_initializer = RandomNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2))(cov'+str(i)+'))')
                                elif weight_initialize_method=='RandomUniform':
                                    exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((cov'+str(i)+'.shape[1]-((cov'+str(i)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((cov'+str(i)+'.shape[1]-((cov'+str(i)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2)),(math.floor(math.floor((cov'+str(i)+'.shape[2]-((cov'+str(i)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((cov'+str(i)+'.shape[2]-((cov'+str(i)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2))))(LocallyConnected2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,kernel_initializer = RandomUniform(minval=weight_initialize_parameter1,maxval=weight_initialize_parameter2))(cov'+str(i)+'))')
                                elif weight_initialize_method=='TruncatedNormal':
                                    exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((cov'+str(i)+'.shape[1]-((cov'+str(i)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((cov'+str(i)+'.shape[1]-((cov'+str(i)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2)),(math.floor(math.floor((cov'+str(i)+'.shape[2]-((cov'+str(i)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((cov'+str(i)+'.shape[2]-((cov'+str(i)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2))))(LocallyConnected2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,kernel_initializer = TruncatedNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2,metrics="MeanSquaredError"))(cov'+str(i)+'))')
                    if ifnormalization=='yes':
                        if normalization_method=='batchnormalization':
                            exec('norm'+str(i+1)+'=BatchNormalization(axis=-1)(cov'+str(i+1)+')')
                        elif normalization_method=='layernormalization':
                            exec('norm'+str(i+1)+'=LayerNormalization(axis=-1)(cov'+str(i+1)+')')
                    if ifnormalization=='yes':
                        covup=eval('norm'+str(k))
                        covdown=eval('norm'+str(l))
                    else:
                        covup=eval('cov'+str(k))
                        covdown=eval('cov'+str(l))
                    exec('up'+str(i+1)+'=concatenate([ZeroPadding2D(padding=((math.floor((covdown.shape[1]-covup.shape[1]*upsize)/2),math.ceil((covdown.shape[1]-covup.shape[1]*upsize)/2)),(math.floor((covdown.shape[2]-covup.shape[2]*upsize)/2),math.ceil((covdown.shape[2]-covup.shape[2]*upsize)/2))))(UpSampling2D(size=(upsize, upsize))(covup)), covdown], axis=3)')
                    for j in range(int(covnum)):
                        if cov_method=='cov':
                            if if_weight_initialize=='no':
                                if j==0:
                                    exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((up'+str(i+1)+'.shape[1]-(up'+str(i+1)+'.shape[1]/cov_strides)))/2),math.ceil(math.floor((up'+str(i+1)+'.shape[1]-(up'+str(i+1)+'.shape[1]/cov_strides)))/2)),(math.floor(math.floor((up'+str(i+1)+'.shape[2]-(up'+str(i+1)+'.shape[2]/cov_strides)))/2),math.ceil(math.floor((up'+str(i+1)+'.shape[2]-(up'+str(i+1)+'.shape[2]/cov_strides)))/2))))(Conv2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,padding="same")(up'+str(i+1)+'))')
                                else:
                                    if ifnormalization=='yes':
                                        exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((norm'+str(i+1)+'.shape[1]-(norm'+str(i+1)+'.shape[1]/cov_strides)))/2),math.ceil(math.floor((norm'+str(i+1)+'.shape[1]-(norm'+str(i+1)+'.shape[1]/cov_strides)))/2)),(math.floor(math.floor((norm'+str(i+1)+'.shape[2]-(norm'+str(i+1)+'.shape[2]/cov_strides)))/2),math.ceil(math.floor((norm'+str(i+1)+'.shape[2]-(norm'+str(i+1)+'.shape[2]/cov_strides)))/2))))(Conv2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,padding="same")(norm'+str(i+1)+'))')
                                    else:
                                        exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((cov'+str(i+1)+'.shape[1]-(cov'+str(i+1)+'.shape[1]/cov_strides)))/2),math.ceil(math.floor((cov'+str(i+1)+'.shape[1]-(cov'+str(i+1)+'.shape[1]/cov_strides)))/2)),(math.floor(math.floor((cov'+str(i+1)+'.shape[2]-(cov'+str(i+1)+'.shape[2]/cov_strides)))/2),math.ceil(math.floor((cov'+str(i+1)+'.shape[2]-(cov'+str(i+1)+'.shape[2]/cov_strides)))/2))))(Conv2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,padding="same")(cov'+str(i+1)+'))')
                            elif if_weight_initialize=='yes':
                                if j==0:
                                    if weight_initialize_method=='RandomNormal':
                                        exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((up'+str(i+1)+'.shape[1]-(up'+str(i+1)+'.shape[1]/cov_strides)))/2),math.ceil(math.floor((up'+str(i+1)+'.shape[1]-(up'+str(i+1)+'.shape[1]/cov_strides)))/2)),(math.floor(math.floor((up'+str(i+1)+'.shape[2]-(up'+str(i+1)+'.shape[2]/cov_strides)))/2),math.ceil(math.floor((up'+str(i+1)+'.shape[2]-(up'+str(i+1)+'.shape[2]/cov_strides)))/2))))(Conv2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,padding="same",kernel_initializer = RandomNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2))(up'+str(i+1)+'))')
                                    elif weight_initialize_method=='RandomUniform':
                                        exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((up'+str(i+1)+'.shape[1]-(up'+str(i+1)+'.shape[1]/cov_strides)))/2),math.ceil(math.floor((up'+str(i+1)+'.shape[1]-(up'+str(i+1)+'.shape[1]/cov_strides)))/2)),(math.floor(math.floor((up'+str(i+1)+'.shape[2]-(up'+str(i+1)+'.shape[2]/cov_strides)))/2),math.ceil(math.floor((up'+str(i+1)+'.shape[2]-(up'+str(i+1)+'.shape[2]/cov_strides)))/2))))(Conv2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,padding="same",kernel_initializer = RandomUniform(minval=weight_initialize_parameter1,maxval=weight_initialize_parameter2))(up'+str(i+1)+'))')
                                    elif weight_initialize_method=='TruncatedNormal':
                                        exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((up'+str(i+1)+'.shape[1]-(up'+str(i+1)+'.shape[1]/cov_strides)))/2),math.ceil(math.floor((up'+str(i+1)+'.shape[1]-(up'+str(i+1)+'.shape[1]/cov_strides)))/2)),(math.floor(math.floor((up'+str(i+1)+'.shape[2]-(up'+str(i+1)+'.shape[2]/cov_strides)))/2),math.ceil(math.floor((up'+str(i+1)+'.shape[2]-(up'+str(i+1)+'.shape[2]/cov_strides)))/2))))(Conv2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,padding="same",kernel_initializer = TruncatedNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2,metrics="MeanSquaredError"))(up'+str(i+1)+'))')
                                else:
                                    if ifnormalization=='yes':
                                        if weight_initialize_method=='RandomNormal':
                                            exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((norm'+str(i+1)+'.shape[1]-(norm'+str(i+1)+'.shape[1]/cov_strides)))/2),math.ceil(math.floor((norm'+str(i+1)+'.shape[1]-(norm'+str(i+1)+'.shape[1]/cov_strides)))/2)),(math.floor(math.floor((norm'+str(i+1)+'.shape[2]-(norm'+str(i+1)+'.shape[2]/cov_strides)))/2),math.ceil(math.floor((norm'+str(i+1)+'.shape[2]-(norm'+str(i+1)+'.shape[2]/cov_strides)))/2))))(Conv2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,padding="same",kernel_initializer = RandomNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2))(norm'+str(i+1)+'))')
                                        elif weight_initialize_method=='RandomUniform':
                                            exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((norm'+str(i+1)+'.shape[1]-(norm'+str(i+1)+'.shape[1]/cov_strides)))/2),math.ceil(math.floor((norm'+str(i+1)+'.shape[1]-(norm'+str(i+1)+'.shape[1]/cov_strides)))/2)),(math.floor(math.floor((norm'+str(i+1)+'.shape[2]-(norm'+str(i+1)+'.shape[2]/cov_strides)))/2),math.ceil(math.floor((norm'+str(i+1)+'.shape[2]-(norm'+str(i+1)+'.shape[2]/cov_strides)))/2))))(Conv2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,padding="same",kernel_initializer = RandomUniform(minval=weight_initialize_parameter1,maxval=weight_initialize_parameter2))(norm'+str(i+1)+'))')
                                        elif weight_initialize_method=='TruncatedNormal':
                                            exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((norm'+str(i+1)+'.shape[1]-(norm'+str(i+1)+'.shape[1]/cov_strides)))/2),math.ceil(math.floor((norm'+str(i+1)+'.shape[1]-(norm'+str(i+1)+'.shape[1]/cov_strides)))/2)),(math.floor(math.floor((norm'+str(i+1)+'.shape[2]-(norm'+str(i+1)+'.shape[2]/cov_strides)))/2),math.ceil(math.floor((norm'+str(i+1)+'.shape[2]-(norm'+str(i+1)+'.shape[2]/cov_strides)))/2))))(Conv2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,padding="same",kernel_initializer = TruncatedNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2,metrics="MeanSquaredError"))(norm'+str(i+1)+'))')
                                    else:
                                        if weight_initialize_method=='RandomNormal':
                                            exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((cov'+str(i+1)+'.shape[1]-(cov'+str(i+1)+'.shape[1]/cov_strides)))/2),math.ceil(math.floor((cov'+str(i+1)+'.shape[1]-(cov'+str(i+1)+'.shape[1]/cov_strides)))/2)),(math.floor(math.floor((cov'+str(i+1)+'.shape[2]-(cov'+str(i+1)+'.shape[2]/cov_strides)))/2),math.ceil(math.floor((cov'+str(i+1)+'.shape[2]-(cov'+str(i+1)+'.shape[2]/cov_strides)))/2))))(Conv2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,padding="same",kernel_initializer = RandomNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2))(cov'+str(i+1)+'))')
                                        elif weight_initialize_method=='RandomUniform':
                                            exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((cov'+str(i+1)+'.shape[1]-(cov'+str(i+1)+'.shape[1]/cov_strides)))/2),math.ceil(math.floor((cov'+str(i+1)+'.shape[1]-(cov'+str(i+1)+'.shape[1]/cov_strides)))/2)),(math.floor(math.floor((cov'+str(i+1)+'.shape[2]-(cov'+str(i+1)+'.shape[2]/cov_strides)))/2),math.ceil(math.floor((cov'+str(i+1)+'.shape[2]-(cov'+str(i+1)+'.shape[2]/cov_strides)))/2))))(Conv2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,padding="same",kernel_initializer = RandomUniform(minval=weight_initialize_parameter1,maxval=weight_initialize_parameter2))(cov'+str(i+1)+'))')
                                        elif weight_initialize_method=='TruncatedNormal':
                                            exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((cov'+str(i+1)+'.shape[1]-(cov'+str(i+1)+'.shape[1]/cov_strides)))/2),math.ceil(math.floor((cov'+str(i+1)+'.shape[1]-(cov'+str(i+1)+'.shape[1]/cov_strides)))/2)),(math.floor(math.floor((cov'+str(i+1)+'.shape[2]-(cov'+str(i+1)+'.shape[2]/cov_strides)))/2),math.ceil(math.floor((cov'+str(i+1)+'.shape[2]-(cov'+str(i+1)+'.shape[2]/cov_strides)))/2))))(Conv2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,padding="same",kernel_initializer = TruncatedNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2,metrics="MeanSquaredError"))(cov'+str(i+1)+'))')
                        elif cov_method=='localcov':
                            if if_weight_initialize=='no':
                                if j==0:
                                    exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((up'+str(i+1)+'.shape[1]-((up'+str(i+1)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((up'+str(i+1)+'.shape[1]-((up'+str(i+1)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2)),(math.floor(math.floor((up'+str(i+1)+'.shape[2]-((up'+str(i+1)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((up'+str(i+1)+'.shape[2]-((up'+str(i+1)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2))))(LocallyConnected2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate)(up'+str(i+1)+'))')
                                else:
                                    if ifnormalization=='yes':
                                        exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((cov'+str(i+1)+'.shape[1]-((cov'+str(i+1)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((cov'+str(i+1)+'.shape[1]-((cov'+str(i+1)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2)),(math.floor(math.floor((cov'+str(i+1)+'.shape[2]-((cov'+str(i+1)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((cov'+str(i+1)+'.shape[2]-((cov'+str(i+1)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2))))(LocallyConnected2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate)(norm'+str(i+1)+'))')
                                    else:
                                        exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((cov'+str(i+1)+'.shape[1]-((cov'+str(i+1)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((cov'+str(i+1)+'.shape[1]-((cov'+str(i+1)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2)),(math.floor(math.floor((cov'+str(i+1)+'.shape[2]-((cov'+str(i+1)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((cov'+str(i+1)+'.shape[2]-((cov'+str(i+1)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2))))(LocallyConnected2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate)(cov'+str(i+1)+'))')
                            elif if_weight_initialize=='yes':
                                if j==0:
                                    if weight_initialize_method=='RandomNormal':
                                        exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((up'+str(i+1)+'.shape[1]-((up'+str(i+1)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((up'+str(i+1)+'.shape[1]-((up'+str(i+1)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2)),(math.floor(math.floor((up'+str(i+1)+'.shape[2]-((up'+str(i+1)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((up'+str(i+1)+'.shape[2]-((up'+str(i+1)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2))))(LocallyConnected2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,kernel_initializer = RandomNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2))(up'+str(i+1)+'))')
                                    elif weight_initialize_method=='RandomUniform':
                                        exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((up'+str(i+1)+'.shape[1]-((up'+str(i+1)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((up'+str(i+1)+'.shape[1]-((up'+str(i+1)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2)),(math.floor(math.floor((up'+str(i+1)+'.shape[2]-((up'+str(i+1)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((up'+str(i+1)+'.shape[2]-((up'+str(i+1)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2))))(LocallyConnected2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,kernel_initializer = RandomUniform(minval=weight_initialize_parameter1,maxval=weight_initialize_parameter2))(up'+str(i+1)+'))')
                                    elif weight_initialize_method=='TruncatedNormal':
                                        exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((up'+str(i+1)+'.shape[1]-((up'+str(i+1)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((up'+str(i+1)+'.shape[1]-((up'+str(i+1)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2)),(math.floor(math.floor((up'+str(i+1)+'.shape[2]-((up'+str(i+1)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((up'+str(i+1)+'.shape[2]-((up'+str(i+1)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2))))(LocallyConnected2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,kernel_initializer = TruncatedNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2,metrics="MeanSquaredError"))(up'+str(i+1)+'))')
                                else:
                                    if ifnormalization=='yes':
                                        if weight_initialize_method=='RandomNormal':
                                            exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((norm'+str(i+1)+'.shape[1]-((norm'+str(i+1)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((norm'+str(i+1)+'.shape[1]-((norm'+str(i+1)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2)),(math.floor(math.floor((norm'+str(i+1)+'.shape[2]-((norm'+str(i+1)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((norm'+str(i+1)+'.shape[2]-((norm'+str(i+1)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2))))(LocallyConnected2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,kernel_initializer = RandomNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2))(norm'+str(i+1)+'))')
                                        elif weight_initialize_method=='RandomUniform':
                                            exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((norm'+str(i+1)+'.shape[1]-((norm'+str(i+1)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((norm'+str(i+1)+'.shape[1]-((norm'+str(i+1)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2)),(math.floor(math.floor((norm'+str(i+1)+'.shape[2]-((norm'+str(i+1)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((norm'+str(i+1)+'.shape[2]-((norm'+str(i+1)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2))))(LocallyConnected2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,kernel_initializer = RandomUniform(minval=weight_initialize_parameter1,maxval=weight_initialize_parameter2))(norm'+str(i+1)+'))')
                                        elif weight_initialize_method=='TruncatedNormal':
                                            exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((norm'+str(i+1)+'.shape[1]-((norm'+str(i+1)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((norm'+str(i+1)+'.shape[1]-((norm'+str(i+1)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2)),(math.floor(math.floor((norm'+str(i+1)+'.shape[2]-((norm'+str(i+1)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((norm'+str(i+1)+'.shape[2]-((norm'+str(i+1)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2))))(LocallyConnected2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,kernel_initializer = TruncatedNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2,metrics="MeanSquaredError"))(norm'+str(i+1)+'))')
                                    else:
                                        if weight_initialize_method=='RandomNormal':
                                            exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((cov'+str(i+1)+'.shape[1]-((cov'+str(i+1)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((cov'+str(i+1)+'.shape[1]-((cov'+str(i+1)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2)),(math.floor(math.floor((cov'+str(i+1)+'.shape[2]-((cov'+str(i+1)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((cov'+str(i+1)+'.shape[2]-((cov'+str(i+1)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2))))(LocallyConnected2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,kernel_initializer = RandomNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2))(cov'+str(i+1)+'))')
                                        elif weight_initialize_method=='RandomUniform':
                                            exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((cov'+str(i+1)+'.shape[1]-((cov'+str(i+1)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((cov'+str(i+1)+'.shape[1]-((cov'+str(i+1)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2)),(math.floor(math.floor((cov'+str(i+1)+'.shape[2]-((cov'+str(i+1)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((cov'+str(i+1)+'.shape[2]-((cov'+str(i+1)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2))))(LocallyConnected2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,kernel_initializer = RandomUniform(minval=weight_initialize_parameter1,maxval=weight_initialize_parameter2))(cov'+str(i+1)+'))')
                                        elif weight_initialize_method=='TruncatedNormal':
                                            exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((cov'+str(i+1)+'.shape[1]-((cov'+str(i+1)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((cov'+str(i+1)+'.shape[1]-((cov'+str(i+1)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2)),(math.floor(math.floor((cov'+str(i+1)+'.shape[2]-((cov'+str(i+1)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((cov'+str(i+1)+'.shape[2]-((cov'+str(i+1)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2))))(LocallyConnected2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,kernel_initializer = TruncatedNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2,metrics="MeanSquaredError"))(cov'+str(i+1)+'))')
                        if ifnormalization=='yes':
                            if normalization_method=='batchnormalization':
                                exec('norm'+str(i+1)+'=BatchNormalization(axis=-1)(cov'+str(i+1)+')')
                            elif normalization_method=='layernormalization':
                                exec('norm'+str(i+1)+'=LayerNormalization(axis=-1)(cov'+str(i+1)+')')
                    k=k+1
                    l=l-1
                if if_last_act!='no':
                    if ifnormalization=='yes':
                        outputs=eval('Conv2D(vy.shape[3], (1, 1),strides=(1,1),padding="same",activation=if_last_act)(norm'+str(2*downnum-1)+')')
                    else:
                        outputs=eval('Conv2D(vy.shape[3], (1, 1),strides=(1,1),padding="same",activation=if_last_act)(cov'+str(2*downnum-1)+')')
                else:
                    if ifnormalization=='yes':
                        outputs=eval('Conv2D(vy.shape[3], (1, 1),strides=(1,1),padding="same")(norm'+str(2*downnum-1)+')')
                    else:
                        outputs=eval('Conv2D(vy.shape[3], (1, 1),strides=(1,1),padding="same")(cov'+str(2*downnum-1)+')')
                model=eval('Model(inputs=inputs, outputs=outputs)')
                if optimizer == 'SGD':
                    opt = SGD(lr = learning_rate)
                elif optimizer == 'Adam':
                    opt = Adam(lr = learning_rate)
                model.compile(loss=loss,optimizer=opt,metrics=[metric])
            elif if_best_mode=='yes' or if_best_mode=='load':
                model=load_model(modelpath)
            trainx=np.nan_to_num(trainx,nan=0)
            testx=np.nan_to_num(testx,nan=0)
            trainy=np.nan_to_num(trainy,nan=0)
            testy=np.nan_to_num(testy,nan=0)
            if if_print_model=='yes':
                print(model.summary())
            if epochs!=0:
                H = model.fit(trainx,trainy,validation_data=(testx,testy),epochs = epochs,batch_size=batch_size)
            r=np.zeros((testx.shape[1],testx.shape[2],testy.shape[3]))
            p=np.zeros((testx.shape[1],testx.shape[2],testy.shape[3]))
            f1score=np.zeros((testx.shape[1],testx.shape[2],testy.shape[3]))
            accuracyscore=np.zeros((testx.shape[1],testx.shape[2],testy.shape[3]))
            recallscore=np.zeros((testx.shape[1],testx.shape[2],testy.shape[3]))
            precisionscore=np.zeros((testx.shape[1],testx.shape[2],testy.shape[3]))
            predicty = model.predict(testx,batch_size=batch_size)
            predicty=predicty.reshape(predicty.shape[0],predicty.shape[1],predicty.shape[2],predicty.shape[3])
            for i in range(testx.shape[1]):
                for j in range(testx.shape[2]):
                    for k in range(testy.shape[3]):
                        if task_mode=='regression':
                            r[i,j,k],p[i,j,k] = pearsonr(predicty[:,i,j,k],testy[:,i,j,k])
                            r[i,j,k]=np.nan_to_num(r[i,j,k],nan=0)
                        elif task_mode=='binary_classify':
                            predicty[:,i,j,k]=[int(round(predicty[l,i,j,k],0)) for l in range(predicty.shape[0])]
                            if metrics=='Recall':
                                r[i,j,k]=recall_score(testy[:,i,j,k], predicty[:,i,j,k])
                            elif metrics=='Precision':
                                r[i,j,k]=precision_score(testy[:,i,j,k], predicty[:,i,j,k])
                            else:
                                r[i,j,k]=accuracy_score(testy[:,i,j,k], predicty[:,i,j,k])
                            p[i,j,k]=0
                            f1score[i,j,k]=f1_score(testy[:,i,j,k],predicty[:,i,j,k])
                            accuracyscore[i,j,k]=accuracy_score(testy[:,i,j,k],predicty[:,i,j,k])
                            recallscore[i,j,k]=recall_score(testy[:,i,j,k],predicty[:,i,j,k])
                            precisionscore=precision_score(testy[:,i,j,k],predicty[:,i,j,k])
                        elif task_mode=='multi_classify':
                            r[i,j,k]=accuracy_score(testy[:,i,j,k], predicty[:,i,j,k])
                            p[i,j,k]=0
            if ifmute == 'no':
                if task_mode=='regression':
                    print('相关系数',np.nanmean(r))
                elif task_mode=='binary_classify':
                    print('召回率+精确率',np.nanmean(f1score),'准确率',np.nanmean(accuracyscore),'召回率',np.nanmean(recallscore),'精确率',np.nanmean(precisionscore))
                elif task_mode=='multi_classify':
                    print('准确率',np.nanmean(r))
            if ifweight=='yes':
                weights=np.zeros((testy.shape[3],testx.shape[3]))
                weight_more=np.zeros((testy.shape[3],testx.shape[3]))
                for i in range(testy.shape[3]):
                    for j in range(testx.shape[3]):
                        testx_new=copy.deepcopy(testx)
                        weight=[]
                        for k in range(10):
                            per=np.random.permutation(testx.shape[0])
                            testx_shuffle=testx[per,:,:,j]
                            testx_new[:,:,:,j]=testx_shuffle
                            predicty_new=np.array(model.predict(testx_new,batch_size=batch_size))
                            for l in range(testy.shape[1]):
                                for m in range(testy.shape[2]):
                                    if task_mode=='regression':
                                        weight.append(sklearn.metrics.mean_squared_error(testy[:,l,m,i],predicty_new[:,l,m,i])-sklearn.metrics.mean_squared_error(testy[:,l,m,i],predicty[:,l,m,i]))
                                    else:
                                        weight.append(sklearn.metrics.log_loss(testy[:,l,m,i],predicty_new[:,l,m,i])-sklearn.metrics.log_loss(testy[:,l,m,i],predicty[:,l,m,i]))
                        weight_more[i,j]=np.nanmean(weight)
                for i in range(testy.shape[3]):
                    for j in range(testx.shape[3]):
                        weights[i,j]=(weight_more[i,j]/np.nansum(weight_more[i,:]))*100
                        print('预报因子',j+1,'对预报值',i+1,'的贡献：',np.array(weights[i,j]),'％')
                    print('\n')
            if ifsave=='yes':
                model.save(savepath)
    elif device=='gpu':
        model=None
        predicty=None
        testy=None
        r=None
        p=None
        k=0
        total_covnum=0
        if ifrandom_split=='yes':
            trainy,testy,trainx,testx = train_test_split(vy,vx,test_size=test_size,random_state=25)
        else:
            index=int((1-test_size)*vy.shape[0])
            trainy=vy[:index,:,:,:]
            testy=vy[index:,:,:,:]
            trainx=vx[:index,:,:,:]
            testx=vx[index:,:,:,:]
        if if_best_mode=='no':
            W=trainx.shape[1]
            H=trainx.shape[2]
            for i in range(int(downnum-1)):
                W=math.floor(((W-pool_kernel_size)/pool_strides)+1)
                H=math.floor(((H-pool_kernel_size)/pool_strides)+1)
            if W<1 or H<1:
                print('下采样次数过多,导致最下层图形长度/宽度小于1!')
                return model,predicty,testy,r,p
            if pool_strides!=pool_kernel_size:
                print('池化步长和池化核不等,将导致下采样前后通道数相除不为整数！')
                return model,predicty,testy,r,p
            upsize=pool_strides
            inputs=Input((trainx.shape[1],trainx.shape[2],trainx.shape[3]))
            for i in range(int(downnum)):
                for j in range(int(covnum)):
                    if cov_method=='cov':
                        if if_weight_initialize=='no':
                            if total_covnum ==0:
                                exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((trainx.shape[1]-(trainx.shape[1]/cov_strides)))/2),math.ceil(math.floor((trainx.shape[1]-(trainx.shape[1]/cov_strides)))/2)),(math.floor(math.floor((trainx.shape[2]-(trainx.shape[2]/cov_strides)))/2),math.ceil(math.floor((trainx.shape[2]-(trainx.shape[2]/cov_strides)))/2))))(Conv2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,padding="same")(inputs))')
                            else:
                                if j==0:
                                    exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((pool'+str(i)+'.shape[1]-(pool'+str(i)+'.shape[1]/cov_strides)))/2),math.ceil(math.floor((pool'+str(i)+'.shape[1]-(pool'+str(i)+'.shape[1]/cov_strides)))/2)),(math.floor(math.floor((pool'+str(i)+'.shape[2]-(pool'+str(i)+'.shape[2]/cov_strides)))/2),math.ceil(math.floor((pool'+str(i)+'.shape[2]-(pool'+str(i)+'.shape[2]/cov_strides)))/2))))(Conv2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,padding="same")(pool'+str(i)+'))')
                                else:
                                    if ifnormalization=='yes':
                                        exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((norm'+str(i+1)+'.shape[1]-(norm'+str(i+1)+'.shape[1]/cov_strides)))/2),math.ceil(math.floor((norm'+str(i+1)+'.shape[1]-(norm'+str(i+1)+'.shape[1]/cov_strides)))/2)),(math.floor(math.floor((norm'+str(i+1)+'.shape[2]-(norm'+str(i+1)+'.shape[2]/cov_strides)))/2),math.ceil(math.floor((norm'+str(i+1)+'.shape[2]-(norm'+str(i+1)+'.shape[2]/cov_strides)))/2))))(Conv2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,padding="same")(norm'+str(i+1)+'))')
                                    else:
                                        exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((cov'+str(i+1)+'.shape[1]-(cov'+str(i+1)+'.shape[1]/cov_strides)))/2),math.ceil(math.floor((cov'+str(i+1)+'.shape[1]-(cov'+str(i+1)+'.shape[1]/cov_strides)))/2)),(math.floor(math.floor((cov'+str(i+1)+'.shape[2]-(cov'+str(i+1)+'.shape[2]/cov_strides)))/2),math.ceil(math.floor((cov'+str(i+1)+'.shape[2]-(cov'+str(i+1)+'.shape[2]/cov_strides)))/2))))(Conv2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,padding="same")(cov'+str(i+1)+'))')
                        elif if_weight_initialize=='yes':
                            if total_covnum==0:
                                if weight_initialize_method=='RandomNormal':
                                    exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((trainx.shape[1]-(trainx.shape[1]/cov_strides)))/2),math.ceil(math.floor((trainx.shape[1]-(trainx.shape[1]/cov_strides)))/2)),(math.floor(math.floor((trainx.shape[2]-(trainx.shape[2]/cov_strides)))/2),math.ceil(math.floor((trainx.shape[2]-(trainx.shape[2]/cov_strides)))/2))))(Conv2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,padding="same",kernel_initializer = RandomNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2))(inputs))')
                                elif weight_initialize_method=='RandomUniform':
                                    exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((trainx.shape[1]-(trainx.shape[1]/cov_strides)))/2),math.ceil(math.floor((trainx.shape[1]-(trainx.shape[1]/cov_strides)))/2)),(math.floor(math.floor((trainx.shape[2]-(trainx.shape[2]/cov_strides)))/2),math.ceil(math.floor((trainx.shape[2]-(trainx.shape[2]/cov_strides)))/2))))(Conv2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,padding="same",kernel_initializer = RandomUniform(minval=weight_initialize_parameter1,maxval=weight_initialize_parameter2))(inputs))')
                                elif weight_initialize_method=='TruncatedNormal':
                                    exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((trainx.shape[1]-(trainx.shape[1]/cov_strides)))/2),math.ceil(math.floor((trainx.shape[1]-(trainx.shape[1]/cov_strides)))/2)),(math.floor(math.floor((trainx.shape[2]-(trainx.shape[2]/cov_strides)))/2),math.ceil(math.floor((trainx.shape[2]-(trainx.shape[2]/cov_strides)))/2))))(Conv2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,padding="same", kernel_initializer = TruncatedNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2,metrics="MeanSquaredError"))(inputs))')
                            else:
                                if j==0:
                                    if weight_initialize_method=='RandomNormal':
                                        exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((pool'+str(i)+'.shape[1]-(pool'+str(i)+'.shape[1]/cov_strides)))/2),math.ceil(math.floor((pool'+str(i)+'.shape[1]-(pool'+str(i)+'.shape[1]/cov_strides)))/2)),(math.floor(math.floor((pool'+str(i)+'.shape[2]-(pool'+str(i)+'.shape[2]/cov_strides)))/2),math.ceil(math.floor((pool'+str(i)+'.shape[2]-(pool'+str(i)+'.shape[2]/cov_strides)))/2))))(Conv2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,padding="same",kernel_initializer = RandomNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2))(pool'+str(i)+'))')
                                    elif weight_initialize_method=='RandomUniform':
                                        exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((pool'+str(i)+'.shape[1]-(pool'+str(i)+'.shape[1]/cov_strides)))/2),math.ceil(math.floor((pool'+str(i)+'.shape[1]-(pool'+str(i)+'.shape[1]/cov_strides)))/2)),(math.floor(math.floor((pool'+str(i)+'.shape[2]-(pool'+str(i)+'.shape[2]/cov_strides)))/2),math.ceil(math.floor((pool'+str(i)+'.shape[2]-(pool'+str(i)+'.shape[2]/cov_strides)))/2))))(Conv2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,padding="same",kernel_initializer = RandomUniform(minval=weight_initialize_parameter1,maxval=weight_initialize_parameter2))(pool'+str(i)+'))')
                                    elif weight_initialize_method=='TruncatedNormal':
                                        exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((pool'+str(i)+'.shape[1]-(pool'+str(i)+'.shape[1]/cov_strides)))/2),math.ceil(math.floor((pool'+str(i)+'.shape[1]-(pool'+str(i)+'.shape[1]/cov_strides)))/2)),(math.floor(math.floor((pool'+str(i)+'.shape[2]-(pool'+str(i)+'.shape[2]/cov_strides)))/2),math.ceil(math.floor((pool'+str(i)+'.shape[2]-(pool'+str(i)+'.shape[2]/cov_strides)))/2))))(Conv2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,padding="same",kernel_initializer = TruncatedNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2,metrics="MeanSquaredError"))(pool'+str(i)+'))')
                                else:
                                    if ifnormalization=='yes':
                                        if weight_initialize_method=='RandomNormal':
                                            exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((norm'+str(i+1)+'.shape[1]-(norm'+str(i+1)+'.shape[1]/cov_strides)))/2),math.ceil(math.floor((norm'+str(i+1)+'.shape[1]-(norm'+str(i+1)+'.shape[1]/cov_strides)))/2)),(math.floor(math.floor((norm'+str(i+1)+'.shape[2]-(norm'+str(i+1)+'.shape[2]/cov_strides)))/2),math.ceil(math.floor((norm'+str(i+1)+'.shape[2]-(norm'+str(i+1)+'.shape[2]/cov_strides)))/2))))(Conv2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,padding="same",kernel_initializer = RandomNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2))(norm'+str(i+1)+'))')
                                        elif weight_initialize_method=='RandomUniform':
                                            exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((norm'+str(i+1)+'.shape[1]-(norm'+str(i+1)+'.shape[1]/cov_strides)))/2),math.ceil(math.floor((norm'+str(i+1)+'.shape[1]-(norm'+str(i+1)+'.shape[1]/cov_strides)))/2)),(math.floor(math.floor((norm'+str(i+1)+'.shape[2]-(norm'+str(i+1)+'.shape[2]/cov_strides)))/2),math.ceil(math.floor((norm'+str(i+1)+'.shape[2]-(norm'+str(i+1)+'.shape[2]/cov_strides)))/2))))(Conv2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,padding="same",kernel_initializer = RandomUniform(minval=weight_initialize_parameter1,maxval=weight_initialize_parameter2))(norm'+str(i+1)+'))')
                                        elif weight_initialize_method=='TruncatedNormal':
                                            exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((norm'+str(i+1)+'.shape[1]-(norm'+str(i+1)+'.shape[1]/cov_strides)))/2),math.ceil(math.floor((norm'+str(i+1)+'.shape[1]-(norm'+str(i+1)+'.shape[1]/cov_strides)))/2)),(math.floor(math.floor((norm'+str(i+1)+'.shape[2]-(norm'+str(i+1)+'.shape[2]/cov_strides)))/2),math.ceil(math.floor((norm'+str(i+1)+'.shape[2]-(norm'+str(i+1)+'.shape[2]/cov_strides)))/2))))(Conv2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,padding="same",kernel_initializer = TruncatedNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2,metrics="MeanSquaredError"))(norm'+str(i+1)+'))')
                                    else:
                                        if weight_initialize_method=='RandomNormal':
                                            exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((cov'+str(i+1)+'.shape[1]-(cov'+str(i+1)+'.shape[1]/cov_strides)))/2),math.ceil(math.floor((cov'+str(i+1)+'.shape[1]-(cov'+str(i+1)+'.shape[1]/cov_strides)))/2)),(math.floor(math.floor((cov'+str(i+1)+'.shape[2]-(cov'+str(i+1)+'.shape[2]/cov_strides)))/2),math.ceil(math.floor((cov'+str(i+1)+'.shape[2]-(cov'+str(i+1)+'.shape[2]/cov_strides)))/2))))(Conv2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,padding="same",kernel_initializer = RandomNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2))(cov'+str(i+1)+'))')
                                        elif weight_initialize_method=='RandomUniform':
                                            exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((cov'+str(i+1)+'.shape[1]-(cov'+str(i+1)+'.shape[1]/cov_strides)))/2),math.ceil(math.floor((cov'+str(i+1)+'.shape[1]-(cov'+str(i+1)+'.shape[1]/cov_strides)))/2)),(math.floor(math.floor((cov'+str(i+1)+'.shape[2]-(cov'+str(i+1)+'.shape[2]/cov_strides)))/2),math.ceil(math.floor((cov'+str(i+1)+'.shape[2]-(cov'+str(i+1)+'.shape[2]/cov_strides)))/2))))(Conv2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,padding="same",kernel_initializer = RandomUniform(minval=weight_initialize_parameter1,maxval=weight_initialize_parameter2))(cov'+str(i+1)+'))')
                                        elif weight_initialize_method=='TruncatedNormal':
                                            exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((cov'+str(i+1)+'.shape[1]-(cov'+str(i+1)+'.shape[1]/cov_strides)))/2),math.ceil(math.floor((cov'+str(i+1)+'.shape[1]-(cov'+str(i+1)+'.shape[1]/cov_strides)))/2)),(math.floor(math.floor((cov'+str(i+1)+'.shape[2]-(cov'+str(i+1)+'.shape[2]/cov_strides)))/2),math.ceil(math.floor((cov'+str(i+1)+'.shape[2]-(cov'+str(i+1)+'.shape[2]/cov_strides)))/2))))(Conv2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,padding="same",kernel_initializer = TruncatedNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2,metrics="MeanSquaredError"))(cov'+str(i+1)+'))')
                    elif cov_method=='localcov':
                        if if_weight_initialize=='no':
                            if total_covnum==0:
                                exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((trainx.shape[1]-((trainx.shape[1]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((trainx.shape[1]-((trainx.shape[1]-cov_kernelsize+1)/cov_strides)))/2)),(math.floor(math.floor((trainx.shape[2]-((trainx.shape[2]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((trainx.shape[2]-((trainx.shape[2]-cov_kernelsize+1)/cov_strides)))/2))))(LocallyConnected2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate)(inputs))')
                            else:
                                if j==0:
                                    exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((pool'+str(i)+'.shape[1]-((pool'+str(i)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((pool'+str(i)+'.shape[1]-((pool'+str(i)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2)),(math.floor(math.floor((pool'+str(i)+'.shape[2]-((pool'+str(i)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((pool'+str(i)+'.shape[2]-((pool'+str(i)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2))))(LocallyConnected2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate)(pool'+str(i)+'))')
                                else:
                                    if ifnormalization=='yes':
                                        exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((norm'+str(i+1)+'.shape[1]-((norm'+str(i+1)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((norm'+str(i+1)+'.shape[1]-((norm'+str(i+1)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2)),(math.floor(math.floor((norm'+str(i+1)+'.shape[2]-((norm'+str(i+1)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((norm'+str(i+1)+'.shape[2]-((norm'+str(i+1)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2))))(LocallyConnected2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate)(norm'+str(i+1)+'))')
                                    else:
                                        exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((cov'+str(i+1)+'.shape[1]-((cov'+str(i+1)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((cov'+str(i+1)+'.shape[1]-((cov'+str(i+1)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2)),(math.floor(math.floor((cov'+str(i+1)+'.shape[2]-((cov'+str(i+1)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((cov'+str(i+1)+'.shape[2]-((cov'+str(i+1)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2))))(LocallyConnected2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate)(cov'+str(i+1)+'))')
                        elif if_weight_initialize=='yes':
                            if total_covnum==0:
                                if weight_initialize_method=='RandomNormal':
                                    exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((trainx.shape[1]-((trainx.shape[1]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((trainx.shape[1]-((trainx.shape[1]-cov_kernelsize+1)/cov_strides)))/2)),(math.floor(math.floor((trainx.shape[2]-((trainx.shape[2]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((trainx.shape[2]-((trainx.shape[2]-cov_kernelsize+1)/cov_strides)))/2))))(LocallyConnected2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate, kernel_initializer = RandomNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2))(inputs))')
                                elif weight_initialize_method=='RandomUniform':
                                    exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((trainx.shape[1]-((trainx.shape[1]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((trainx.shape[1]-((trainx.shape[1]-cov_kernelsize+1)/cov_strides)))/2)),(math.floor(math.floor((trainx.shape[2]-((trainx.shape[2]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((trainx.shape[2]-((trainx.shape[2]-cov_kernelsize+1)/cov_strides)))/2))))(LocallyConnected2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate, kernel_initializer = RandomUniform(minval=weight_initialize_parameter1,maxval=weight_initialize_parameter2))(inputs))')
                                elif weight_initialize_method=='TruncatedNormal':
                                    exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((trainx.shape[1]-((trainx.shape[1]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((trainx.shape[1]-((trainx.shape[1]-cov_kernelsize+1)/cov_strides)))/2)),(math.floor(math.floor((trainx.shape[2]-((trainx.shape[2]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((trainx.shape[2]-((trainx.shape[2]-cov_kernelsize+1)/cov_strides)))/2))))(LocallyConnected2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate, kernel_initializer = TruncatedNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2,metrics="MeanSquaredError"))(inputs))')
                            else:
                                if j==0:
                                    if weight_initialize_method=='RandomNormal':
                                        exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((pool'+str(i)+'.shape[1]-((pool'+str(i)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((pool'+str(i)+'.shape[1]-((pool'+str(i)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2)),(math.floor(math.floor((pool'+str(i)+'.shape[2]-((pool'+str(i)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((pool'+str(i)+'.shape[2]-((pool'+str(i)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2))))(LocallyConnected2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,kernel_initializer = RandomNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2))(pool'+str(i)+'))')
                                    elif weight_initialize_method=='RandomUniform':
                                        exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((pool'+str(i)+'.shape[1]-((pool'+str(i)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((pool'+str(i)+'.shape[1]-((pool'+str(i)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2)),(math.floor(math.floor((pool'+str(i)+'.shape[2]-((pool'+str(i)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((pool'+str(i)+'.shape[2]-((pool'+str(i)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2))))(LocallyConnected2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,kernel_initializer = RandomUniform(minval=weight_initialize_parameter1,maxval=weight_initialize_parameter2))(pool'+str(i)+'))')
                                    elif weight_initialize_method=='TruncatedNormal':
                                        exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((pool'+str(i)+'.shape[1]-((pool'+str(i)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((pool'+str(i)+'.shape[1]-((pool'+str(i)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2)),(math.floor(math.floor((pool'+str(i)+'.shape[2]-((pool'+str(i)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((pool'+str(i)+'.shape[2]-((pool'+str(i)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2))))(LocallyConnected2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,kernel_initializer = TruncatedNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2,metrics="MeanSquaredError"))(pool'+str(i)+'))')
                                else:
                                    if ifnormalization=='yes':
                                        if weight_initialize_method=='RandomNormal':
                                            exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((norm'+str(i+1)+'.shape[1]-((norm'+str(i+1)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((norm'+str(i+1)+'.shape[1]-((norm'+str(i+1)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2)),(math.floor(math.floor((norm'+str(i+1)+'.shape[2]-((norm'+str(i+1)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((norm'+str(i+1)+'.shape[2]-((norm'+str(i+1)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2))))(LocallyConnected2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,kernel_initializer = RandomNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2))(norm'+str(i+1)+'))')
                                        elif weight_initialize_method=='RandomUniform':
                                            exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((norm'+str(i+1)+'.shape[1]-((norm'+str(i+1)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((norm'+str(i+1)+'.shape[1]-((norm'+str(i+1)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2)),(math.floor(math.floor((norm'+str(i+1)+'.shape[2]-((norm'+str(i+1)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((norm'+str(i+1)+'.shape[2]-((norm'+str(i+1)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2))))(LocallyConnected2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,kernel_initializer = RandomUniform(minval=weight_initialize_parameter1,maxval=weight_initialize_parameter2))(norm'+str(i+1)+'))')
                                        elif weight_initialize_method=='TruncatedNormal':
                                            exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((norm'+str(i+1)+'.shape[1]-((norm'+str(i+1)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((norm'+str(i+1)+'.shape[1]-((norm'+str(i+1)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2)),(math.floor(math.floor((norm'+str(i+1)+'.shape[2]-((norm'+str(i+1)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((norm'+str(i+1)+'.shape[2]-((norm'+str(i+1)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2))))(LocallyConnected2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,kernel_initializer = TruncatedNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2,metrics="MeanSquaredError"))(norm'+str(i+1)+'))')
                                    else:
                                        if weight_initialize_method=='RandomNormal':
                                            exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((cov'+str(i+1)+'.shape[1]-((cov'+str(i+1)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((cov'+str(i+1)+'.shape[1]-((cov'+str(i+1)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2)),(math.floor(math.floor((cov'+str(i+1)+'.shape[2]-((cov'+str(i+1)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((cov'+str(i+1)+'.shape[2]-((cov'+str(i+1)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2))))(LocallyConnected2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,kernel_initializer = RandomNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2))(cov'+str(i+1)+'))')
                                        elif weight_initialize_method=='RandomUniform':
                                            exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((cov'+str(i+1)+'.shape[1]-((cov'+str(i+1)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((cov'+str(i+1)+'.shape[1]-((cov'+str(i+1)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2)),(math.floor(math.floor((cov'+str(i+1)+'.shape[2]-((cov'+str(i+1)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((cov'+str(i+1)+'.shape[2]-((cov'+str(i+1)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2))))(LocallyConnected2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,kernel_initializer = RandomUniform(minval=weight_initialize_parameter1,maxval=weight_initialize_parameter2))(cov'+str(i+1)+'))')
                                        elif weight_initialize_method=='TruncatedNormal':
                                            exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((cov'+str(i+1)+'.shape[1]-((cov'+str(i+1)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((cov'+str(i+1)+'.shape[1]-((cov'+str(i+1)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2)),(math.floor(math.floor((cov'+str(i+1)+'.shape[2]-((cov'+str(i+1)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((cov'+str(i+1)+'.shape[2]-((cov'+str(i+1)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2))))(LocallyConnected2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,kernel_initializer = TruncatedNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2,metrics="MeanSquaredError"))(cov'+str(i+1)+'))')
                    total_covnum=total_covnum+1
                    if ifnormalization=='yes':
                        if normalization_method=='batchnormalization':
                            exec('norm'+str(i+1)+'=BatchNormalization(axis=-1)(cov'+str(i+1)+')')
                        elif normalization_method=='layernormalization':
                            exec('norm'+str(i+1)+'=LayerNormalization(axis=-1)(cov'+str(i+1)+')')
                k=k+1
                if i!=(downnum-1):
                    baselayer=baselayer*2
                if pool_method=='maxpooling' and i!=(downnum-1):
                    if ifnormalization=='yes':
                        exec('pool'+str(i+1)+'=MaxPooling2D(pool_size=(pool_kernel_size,pool_kernel_size),strides=pool_strides)(norm'+str(i+1)+')')
                    else:
                        exec('pool'+str(i+1)+'=MaxPooling2D(pool_size=(pool_kernel_size,pool_kernel_size),strides=pool_strides)(cov'+str(i+1)+')')
                elif pool_method=='avepooling' and i!=(downnum-1):
                    if ifnormalization=='yes':
                        exec('pool'+str(i+1)+'=AveragePooling2D(pool_size=(pool_kernel_size,pool_kernel_size),strides=pool_strides)(norm'+str(i+1)+')')
                    else:
                        exec('pool'+str(i+1)+'=AveragePooling2D(pool_size=(pool_kernel_size,pool_kernel_size),strides=pool_strides)(cov'+str(i+1)+')')
            l=k-1
            k=k+1
            for i in range(downnum,2*downnum-1):
                baselayer=int(baselayer/2)
                if cov_method=='cov':
                    if if_weight_initialize=='no':
                        if ifnormalization=='yes':
                            exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((norm'+str(i)+'.shape[1]-(norm'+str(i)+'.shape[1]/cov_strides)))/2),math.ceil(math.floor((norm'+str(i)+'.shape[1]-(norm'+str(i)+'.shape[1]/cov_strides)))/2)),(math.floor(math.floor((norm'+str(i)+'.shape[2]-(norm'+str(i)+'.shape[2]/cov_strides)))/2),math.ceil(math.floor((norm'+str(i)+'.shape[2]-(norm'+str(i)+'.shape[2]/cov_strides)))/2))))(Conv2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,padding="same")(norm'+str(i)+'))')
                        else:
                            exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((cov'+str(i)+'.shape[1]-(cov'+str(i)+'.shape[1]/cov_strides)))/2),math.ceil(math.floor((cov'+str(i)+'.shape[1]-(cov'+str(i)+'.shape[1]/cov_strides)))/2)),(math.floor(math.floor((cov'+str(i)+'.shape[2]-(cov'+str(i)+'.shape[2]/cov_strides)))/2),math.ceil(math.floor((cov'+str(i)+'.shape[2]-(cov'+str(i)+'.shape[2]/cov_strides)))/2))))(Conv2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,padding="same")(cov'+str(i)+'))')
                    elif if_weight_initialize=='yes':
                        if ifnormalization=='yes':
                            if weight_initialize_method=='RandomNormal':
                                exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((norm'+str(i)+'.shape[1]-(norm'+str(i)+'.shape[1]/cov_strides)))/2),math.ceil(math.floor((norm'+str(i)+'.shape[1]-(norm'+str(i)+'.shape[1]/cov_strides)))/2)),(math.floor(math.floor((norm'+str(i)+'.shape[2]-(norm'+str(i)+'.shape[2]/cov_strides)))/2),math.ceil(math.floor((norm'+str(i)+'.shape[2]-(norm'+str(i)+'.shape[2]/cov_strides)))/2))))(Conv2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,padding="same",kernel_initializer = RandomNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2))(norm'+str(i)+'))')
                            elif weight_initialize_method=='RandomUniform':
                                exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((norm'+str(i)+'.shape[1]-(norm'+str(i)+'.shape[1]/cov_strides)))/2),math.ceil(math.floor((norm'+str(i)+'.shape[1]-(norm'+str(i)+'.shape[1]/cov_strides)))/2)),(math.floor(math.floor((norm'+str(i)+'.shape[2]-(norm'+str(i)+'.shape[2]/cov_strides)))/2),math.ceil(math.floor((norm'+str(i)+'.shape[2]-(norm'+str(i)+'.shape[2]/cov_strides)))/2))))(Conv2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,padding="same",kernel_initializer = RandomUniform(minval=weight_initialize_parameter1,maxval=weight_initialize_parameter2))(norm'+str(i)+'))')
                            elif weight_initialize_method=='TruncatedNormal':
                                exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((norm'+str(i)+'.shape[1]-(norm'+str(i)+'.shape[1]/cov_strides)))/2),math.ceil(math.floor((norm'+str(i)+'.shape[1]-(norm'+str(i)+'.shape[1]/cov_strides)))/2)),(math.floor(math.floor((norm'+str(i)+'.shape[2]-(norm'+str(i)+'.shape[2]/cov_strides)))/2),math.ceil(math.floor((norm'+str(i)+'.shape[2]-(norm'+str(i)+'.shape[2]/cov_strides)))/2))))(Conv2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,padding="same",kernel_initializer = TruncatedNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2,metrics="MeanSquaredError"))(norm'+str(i)+'))')
                        else:
                            if weight_initialize_method=='RandomNormal':
                                exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((cov'+str(i)+'.shape[1]-(cov'+str(i)+'.shape[1]/cov_strides)))/2),math.ceil(math.floor((cov'+str(i)+'.shape[1]-(cov'+str(i)+'.shape[1]/cov_strides)))/2)),(math.floor(math.floor((cov'+str(i)+'.shape[2]-(cov'+str(i)+'.shape[2]/cov_strides)))/2),math.ceil(math.floor((cov'+str(i)+'.shape[2]-(cov'+str(i)+'.shape[2]/cov_strides)))/2))))(Conv2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,padding="same",kernel_initializer = RandomNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2))(cov'+str(i)+'))')
                            elif weight_initialize_method=='RandomUniform':
                                exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((cov'+str(i)+'.shape[1]-(cov'+str(i)+'.shape[1]/cov_strides)))/2),math.ceil(math.floor((cov'+str(i)+'.shape[1]-(cov'+str(i)+'.shape[1]/cov_strides)))/2)),(math.floor(math.floor((cov'+str(i)+'.shape[2]-(cov'+str(i)+'.shape[2]/cov_strides)))/2),math.ceil(math.floor((cov'+str(i)+'.shape[2]-(cov'+str(i)+'.shape[2]/cov_strides)))/2))))(Conv2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,padding="same",kernel_initializer = RandomUniform(minval=weight_initialize_parameter1,maxval=weight_initialize_parameter2))(cov'+str(i)+'))')
                            elif weight_initialize_method=='TruncatedNormal':
                                exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((cov'+str(i)+'.shape[1]-(cov'+str(i)+'.shape[1]/cov_strides)))/2),math.ceil(math.floor((cov'+str(i)+'.shape[1]-(cov'+str(i)+'.shape[1]/cov_strides)))/2)),(math.floor(math.floor((cov'+str(i)+'.shape[2]-(cov'+str(i)+'.shape[2]/cov_strides)))/2),math.ceil(math.floor((cov'+str(i)+'.shape[2]-(cov'+str(i)+'.shape[2]/cov_strides)))/2))))(Conv2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,padding="same",kernel_initializer = TruncatedNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2,metrics="MeanSquaredError"))(cov'+str(i)+'))')
                elif cov_method=='localcov':
                    if if_weight_initialize=='no':
                        if ifnormalization=='yes':
                            exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((norm'+str(i)+'.shape[1]-((norm'+str(i)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((norm'+str(i)+'.shape[1]-((norm'+str(i)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2)),(math.floor(math.floor((norm'+str(i)+'.shape[2]-((norm'+str(i)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((norm'+str(i)+'.shape[2]-((norm'+str(i)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2))))(LocallyConnected2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate)(norm'+str(i)+'))')
                        else:
                            exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((cov'+str(i)+'.shape[1]-((cov'+str(i)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((cov'+str(i)+'.shape[1]-((cov'+str(i)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2)),(math.floor(math.floor((cov'+str(i)+'.shape[2]-((cov'+str(i)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((cov'+str(i)+'.shape[2]-((cov'+str(i)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2))))(LocallyConnected2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate)(cov'+str(i)+'))')
                    elif if_weight_initialize=='yes':
                        if ifnormalization=='yes':
                            if weight_initialize_method=='RandomNormal':
                                exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((norm'+str(i)+'.shape[1]-((norm'+str(i)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((norm'+str(i)+'.shape[1]-((norm'+str(i)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2)),(math.floor(math.floor((norm'+str(i)+'.shape[2]-((norm'+str(i)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((norm'+str(i)+'.shape[2]-((norm'+str(i)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2))))(LocallyConnected2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,kernel_initializer = RandomNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2))(norm'+str(i)+'))')
                            elif weight_initialize_method=='RandomUniform':
                                exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((norm'+str(i)+'.shape[1]-((norm'+str(i)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((norm'+str(i)+'.shape[1]-((norm'+str(i)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2)),(math.floor(math.floor((norm'+str(i)+'.shape[2]-((norm'+str(i)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((norm'+str(i)+'.shape[2]-((norm'+str(i)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2))))(LocallyConnected2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,kernel_initializer = RandomUniform(minval=weight_initialize_parameter1,maxval=weight_initialize_parameter2))(norm'+str(i)+'))')
                            elif weight_initialize_method=='TruncatedNormal':
                                exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((norm'+str(i)+'.shape[1]-((norm'+str(i)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((norm'+str(i)+'.shape[1]-((norm'+str(i)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2)),(math.floor(math.floor((norm'+str(i)+'.shape[2]-((norm'+str(i)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((norm'+str(i)+'.shape[2]-((norm'+str(i)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2))))(LocallyConnected2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,kernel_initializer = TruncatedNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2,metrics="MeanSquaredError"))(norm'+str(i)+'))')
                        else:
                            if weight_initialize_method=='RandomNormal':
                                exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((cov'+str(i)+'.shape[1]-((cov'+str(i)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((cov'+str(i)+'.shape[1]-((cov'+str(i)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2)),(math.floor(math.floor((cov'+str(i)+'.shape[2]-((cov'+str(i)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((cov'+str(i)+'.shape[2]-((cov'+str(i)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2))))(LocallyConnected2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,kernel_initializer = RandomNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2))(cov'+str(i)+'))')
                            elif weight_initialize_method=='RandomUniform':
                                exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((cov'+str(i)+'.shape[1]-((cov'+str(i)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((cov'+str(i)+'.shape[1]-((cov'+str(i)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2)),(math.floor(math.floor((cov'+str(i)+'.shape[2]-((cov'+str(i)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((cov'+str(i)+'.shape[2]-((cov'+str(i)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2))))(LocallyConnected2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,kernel_initializer = RandomUniform(minval=weight_initialize_parameter1,maxval=weight_initialize_parameter2))(cov'+str(i)+'))')
                            elif weight_initialize_method=='TruncatedNormal':
                                exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((cov'+str(i)+'.shape[1]-((cov'+str(i)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((cov'+str(i)+'.shape[1]-((cov'+str(i)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2)),(math.floor(math.floor((cov'+str(i)+'.shape[2]-((cov'+str(i)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((cov'+str(i)+'.shape[2]-((cov'+str(i)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2))))(LocallyConnected2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,kernel_initializer = TruncatedNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2,metrics="MeanSquaredError"))(cov'+str(i)+'))')
                if ifnormalization=='yes':
                    if normalization_method=='batchnormalization':
                        exec('norm'+str(i+1)+'=BatchNormalization(axis=-1)(cov'+str(i+1)+')')
                    elif normalization_method=='layernormalization':
                        exec('norm'+str(i+1)+'=LayerNormalization(axis=-1)(cov'+str(i+1)+')')
                if ifnormalization=='yes':
                    covup=eval('norm'+str(k))
                    covdown=eval('norm'+str(l))
                else:
                    covup=eval('cov'+str(k))
                    covdown=eval('cov'+str(l))
                exec('up'+str(i+1)+'=concatenate([ZeroPadding2D(padding=((math.floor((covdown.shape[1]-covup.shape[1]*upsize)/2),math.ceil((covdown.shape[1]-covup.shape[1]*upsize)/2)),(math.floor((covdown.shape[2]-covup.shape[2]*upsize)/2),math.ceil((covdown.shape[2]-covup.shape[2]*upsize)/2))))(UpSampling2D(size=(upsize, upsize))(covup)), covdown], axis=3)')
                for j in range(int(covnum)):
                    if cov_method=='cov':
                        if if_weight_initialize=='no':
                            if j==0:
                                exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((up'+str(i+1)+'.shape[1]-(up'+str(i+1)+'.shape[1]/cov_strides)))/2),math.ceil(math.floor((up'+str(i+1)+'.shape[1]-(up'+str(i+1)+'.shape[1]/cov_strides)))/2)),(math.floor(math.floor((up'+str(i+1)+'.shape[2]-(up'+str(i+1)+'.shape[2]/cov_strides)))/2),math.ceil(math.floor((up'+str(i+1)+'.shape[2]-(up'+str(i+1)+'.shape[2]/cov_strides)))/2))))(Conv2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,padding="same")(up'+str(i+1)+'))')
                            else:
                                if ifnormalization=='yes':
                                    exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((norm'+str(i+1)+'.shape[1]-(norm'+str(i+1)+'.shape[1]/cov_strides)))/2),math.ceil(math.floor((norm'+str(i+1)+'.shape[1]-(norm'+str(i+1)+'.shape[1]/cov_strides)))/2)),(math.floor(math.floor((norm'+str(i+1)+'.shape[2]-(norm'+str(i+1)+'.shape[2]/cov_strides)))/2),math.ceil(math.floor((norm'+str(i+1)+'.shape[2]-(norm'+str(i+1)+'.shape[2]/cov_strides)))/2))))(Conv2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,padding="same")(norm'+str(i+1)+'))')
                                else:
                                    exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((cov'+str(i+1)+'.shape[1]-(cov'+str(i+1)+'.shape[1]/cov_strides)))/2),math.ceil(math.floor((cov'+str(i+1)+'.shape[1]-(cov'+str(i+1)+'.shape[1]/cov_strides)))/2)),(math.floor(math.floor((cov'+str(i+1)+'.shape[2]-(cov'+str(i+1)+'.shape[2]/cov_strides)))/2),math.ceil(math.floor((cov'+str(i+1)+'.shape[2]-(cov'+str(i+1)+'.shape[2]/cov_strides)))/2))))(Conv2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,padding="same")(cov'+str(i+1)+'))')
                        elif if_weight_initialize=='yes':
                            if j==0:
                                if weight_initialize_method=='RandomNormal':
                                    exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((up'+str(i+1)+'.shape[1]-(up'+str(i+1)+'.shape[1]/cov_strides)))/2),math.ceil(math.floor((up'+str(i+1)+'.shape[1]-(up'+str(i+1)+'.shape[1]/cov_strides)))/2)),(math.floor(math.floor((up'+str(i+1)+'.shape[2]-(up'+str(i+1)+'.shape[2]/cov_strides)))/2),math.ceil(math.floor((up'+str(i+1)+'.shape[2]-(up'+str(i+1)+'.shape[2]/cov_strides)))/2))))(Conv2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,padding="same",kernel_initializer = RandomNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2))(up'+str(i+1)+'))')
                                elif weight_initialize_method=='RandomUniform':
                                    exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((up'+str(i+1)+'.shape[1]-(up'+str(i+1)+'.shape[1]/cov_strides)))/2),math.ceil(math.floor((up'+str(i+1)+'.shape[1]-(up'+str(i+1)+'.shape[1]/cov_strides)))/2)),(math.floor(math.floor((up'+str(i+1)+'.shape[2]-(up'+str(i+1)+'.shape[2]/cov_strides)))/2),math.ceil(math.floor((up'+str(i+1)+'.shape[2]-(up'+str(i+1)+'.shape[2]/cov_strides)))/2))))(Conv2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,padding="same",kernel_initializer = RandomUniform(minval=weight_initialize_parameter1,maxval=weight_initialize_parameter2))(up'+str(i+1)+'))')
                                elif weight_initialize_method=='TruncatedNormal':
                                    exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((up'+str(i+1)+'.shape[1]-(up'+str(i+1)+'.shape[1]/cov_strides)))/2),math.ceil(math.floor((up'+str(i+1)+'.shape[1]-(up'+str(i+1)+'.shape[1]/cov_strides)))/2)),(math.floor(math.floor((up'+str(i+1)+'.shape[2]-(up'+str(i+1)+'.shape[2]/cov_strides)))/2),math.ceil(math.floor((up'+str(i+1)+'.shape[2]-(up'+str(i+1)+'.shape[2]/cov_strides)))/2))))(Conv2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,padding="same",kernel_initializer = TruncatedNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2,metrics="MeanSquaredError"))(up'+str(i+1)+'))')
                            else:
                                if ifnormalization=='yes':
                                    if weight_initialize_method=='RandomNormal':
                                        exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((norm'+str(i+1)+'.shape[1]-(norm'+str(i+1)+'.shape[1]/cov_strides)))/2),math.ceil(math.floor((norm'+str(i+1)+'.shape[1]-(norm'+str(i+1)+'.shape[1]/cov_strides)))/2)),(math.floor(math.floor((norm'+str(i+1)+'.shape[2]-(norm'+str(i+1)+'.shape[2]/cov_strides)))/2),math.ceil(math.floor((norm'+str(i+1)+'.shape[2]-(norm'+str(i+1)+'.shape[2]/cov_strides)))/2))))(Conv2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,padding="same",kernel_initializer = RandomNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2))(norm'+str(i+1)+'))')
                                    elif weight_initialize_method=='RandomUniform':
                                        exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((norm'+str(i+1)+'.shape[1]-(norm'+str(i+1)+'.shape[1]/cov_strides)))/2),math.ceil(math.floor((norm'+str(i+1)+'.shape[1]-(norm'+str(i+1)+'.shape[1]/cov_strides)))/2)),(math.floor(math.floor((norm'+str(i+1)+'.shape[2]-(norm'+str(i+1)+'.shape[2]/cov_strides)))/2),math.ceil(math.floor((norm'+str(i+1)+'.shape[2]-(norm'+str(i+1)+'.shape[2]/cov_strides)))/2))))(Conv2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,padding="same",kernel_initializer = RandomUniform(minval=weight_initialize_parameter1,maxval=weight_initialize_parameter2))(norm'+str(i+1)+'))')
                                    elif weight_initialize_method=='TruncatedNormal':
                                        exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((norm'+str(i+1)+'.shape[1]-(norm'+str(i+1)+'.shape[1]/cov_strides)))/2),math.ceil(math.floor((norm'+str(i+1)+'.shape[1]-(norm'+str(i+1)+'.shape[1]/cov_strides)))/2)),(math.floor(math.floor((norm'+str(i+1)+'.shape[2]-(norm'+str(i+1)+'.shape[2]/cov_strides)))/2),math.ceil(math.floor((norm'+str(i+1)+'.shape[2]-(norm'+str(i+1)+'.shape[2]/cov_strides)))/2))))(Conv2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,padding="same",kernel_initializer = TruncatedNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2,metrics="MeanSquaredError"))(norm'+str(i+1)+'))')
                                else:
                                    if weight_initialize_method=='RandomNormal':
                                        exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((cov'+str(i+1)+'.shape[1]-(cov'+str(i+1)+'.shape[1]/cov_strides)))/2),math.ceil(math.floor((cov'+str(i+1)+'.shape[1]-(cov'+str(i+1)+'.shape[1]/cov_strides)))/2)),(math.floor(math.floor((cov'+str(i+1)+'.shape[2]-(cov'+str(i+1)+'.shape[2]/cov_strides)))/2),math.ceil(math.floor((cov'+str(i+1)+'.shape[2]-(cov'+str(i+1)+'.shape[2]/cov_strides)))/2))))(Conv2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,padding="same",kernel_initializer = RandomNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2))(cov'+str(i+1)+'))')
                                    elif weight_initialize_method=='RandomUniform':
                                        exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((cov'+str(i+1)+'.shape[1]-(cov'+str(i+1)+'.shape[1]/cov_strides)))/2),math.ceil(math.floor((cov'+str(i+1)+'.shape[1]-(cov'+str(i+1)+'.shape[1]/cov_strides)))/2)),(math.floor(math.floor((cov'+str(i+1)+'.shape[2]-(cov'+str(i+1)+'.shape[2]/cov_strides)))/2),math.ceil(math.floor((cov'+str(i+1)+'.shape[2]-(cov'+str(i+1)+'.shape[2]/cov_strides)))/2))))(Conv2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,padding="same",kernel_initializer = RandomUniform(minval=weight_initialize_parameter1,maxval=weight_initialize_parameter2))(cov'+str(i+1)+'))')
                                    elif weight_initialize_method=='TruncatedNormal':
                                        exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((cov'+str(i+1)+'.shape[1]-(cov'+str(i+1)+'.shape[1]/cov_strides)))/2),math.ceil(math.floor((cov'+str(i+1)+'.shape[1]-(cov'+str(i+1)+'.shape[1]/cov_strides)))/2)),(math.floor(math.floor((cov'+str(i+1)+'.shape[2]-(cov'+str(i+1)+'.shape[2]/cov_strides)))/2),math.ceil(math.floor((cov'+str(i+1)+'.shape[2]-(cov'+str(i+1)+'.shape[2]/cov_strides)))/2))))(Conv2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,padding="same",kernel_initializer = TruncatedNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2,metrics="MeanSquaredError"))(cov'+str(i+1)+'))')
                    elif cov_method=='localcov':
                        if if_weight_initialize=='no':
                            if j==0:
                                exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((up'+str(i+1)+'.shape[1]-((up'+str(i+1)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((up'+str(i+1)+'.shape[1]-((up'+str(i+1)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2)),(math.floor(math.floor((up'+str(i+1)+'.shape[2]-((up'+str(i+1)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((up'+str(i+1)+'.shape[2]-((up'+str(i+1)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2))))(LocallyConnected2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate)(up'+str(i+1)+'))')
                            else:
                                if ifnormalization=='yes':
                                    exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((cov'+str(i+1)+'.shape[1]-((cov'+str(i+1)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((cov'+str(i+1)+'.shape[1]-((cov'+str(i+1)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2)),(math.floor(math.floor((cov'+str(i+1)+'.shape[2]-((cov'+str(i+1)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((cov'+str(i+1)+'.shape[2]-((cov'+str(i+1)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2))))(LocallyConnected2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate)(norm'+str(i+1)+'))')
                                else:
                                    exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((cov'+str(i+1)+'.shape[1]-((cov'+str(i+1)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((cov'+str(i+1)+'.shape[1]-((cov'+str(i+1)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2)),(math.floor(math.floor((cov'+str(i+1)+'.shape[2]-((cov'+str(i+1)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((cov'+str(i+1)+'.shape[2]-((cov'+str(i+1)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2))))(LocallyConnected2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate)(cov'+str(i+1)+'))')
                        elif if_weight_initialize=='yes':
                            if j==0:
                                if weight_initialize_method=='RandomNormal':
                                    exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((up'+str(i+1)+'.shape[1]-((up'+str(i+1)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((up'+str(i+1)+'.shape[1]-((up'+str(i+1)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2)),(math.floor(math.floor((up'+str(i+1)+'.shape[2]-((up'+str(i+1)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((up'+str(i+1)+'.shape[2]-((up'+str(i+1)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2))))(LocallyConnected2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,kernel_initializer = RandomNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2))(up'+str(i+1)+'))')
                                elif weight_initialize_method=='RandomUniform':
                                    exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((up'+str(i+1)+'.shape[1]-((up'+str(i+1)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((up'+str(i+1)+'.shape[1]-((up'+str(i+1)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2)),(math.floor(math.floor((up'+str(i+1)+'.shape[2]-((up'+str(i+1)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((up'+str(i+1)+'.shape[2]-((up'+str(i+1)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2))))(LocallyConnected2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,kernel_initializer = RandomUniform(minval=weight_initialize_parameter1,maxval=weight_initialize_parameter2))(up'+str(i+1)+'))')
                                elif weight_initialize_method=='TruncatedNormal':
                                    exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((up'+str(i+1)+'.shape[1]-((up'+str(i+1)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((up'+str(i+1)+'.shape[1]-((up'+str(i+1)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2)),(math.floor(math.floor((up'+str(i+1)+'.shape[2]-((up'+str(i+1)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((up'+str(i+1)+'.shape[2]-((up'+str(i+1)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2))))(LocallyConnected2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,kernel_initializer = TruncatedNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2,metrics="MeanSquaredError"))(up'+str(i+1)+'))')
                            else:
                                if ifnormalization=='yes':
                                    if weight_initialize_method=='RandomNormal':
                                        exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((norm'+str(i+1)+'.shape[1]-((norm'+str(i+1)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((norm'+str(i+1)+'.shape[1]-((norm'+str(i+1)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2)),(math.floor(math.floor((norm'+str(i+1)+'.shape[2]-((norm'+str(i+1)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((norm'+str(i+1)+'.shape[2]-((norm'+str(i+1)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2))))(LocallyConnected2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,kernel_initializer = RandomNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2))(norm'+str(i+1)+'))')
                                    elif weight_initialize_method=='RandomUniform':
                                        exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((norm'+str(i+1)+'.shape[1]-((norm'+str(i+1)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((norm'+str(i+1)+'.shape[1]-((norm'+str(i+1)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2)),(math.floor(math.floor((norm'+str(i+1)+'.shape[2]-((norm'+str(i+1)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((norm'+str(i+1)+'.shape[2]-((norm'+str(i+1)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2))))(LocallyConnected2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,kernel_initializer = RandomUniform(minval=weight_initialize_parameter1,maxval=weight_initialize_parameter2))(norm'+str(i+1)+'))')
                                    elif weight_initialize_method=='TruncatedNormal':
                                        exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((norm'+str(i+1)+'.shape[1]-((norm'+str(i+1)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((norm'+str(i+1)+'.shape[1]-((norm'+str(i+1)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2)),(math.floor(math.floor((norm'+str(i+1)+'.shape[2]-((norm'+str(i+1)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((norm'+str(i+1)+'.shape[2]-((norm'+str(i+1)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2))))(LocallyConnected2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,kernel_initializer = TruncatedNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2,metrics="MeanSquaredError"))(norm'+str(i+1)+'))')
                                else:
                                    if weight_initialize_method=='RandomNormal':
                                        exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((cov'+str(i+1)+'.shape[1]-((cov'+str(i+1)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((cov'+str(i+1)+'.shape[1]-((cov'+str(i+1)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2)),(math.floor(math.floor((cov'+str(i+1)+'.shape[2]-((cov'+str(i+1)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((cov'+str(i+1)+'.shape[2]-((cov'+str(i+1)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2))))(LocallyConnected2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,kernel_initializer = RandomNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2))(cov'+str(i+1)+'))')
                                    elif weight_initialize_method=='RandomUniform':
                                        exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((cov'+str(i+1)+'.shape[1]-((cov'+str(i+1)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((cov'+str(i+1)+'.shape[1]-((cov'+str(i+1)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2)),(math.floor(math.floor((cov'+str(i+1)+'.shape[2]-((cov'+str(i+1)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((cov'+str(i+1)+'.shape[2]-((cov'+str(i+1)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2))))(LocallyConnected2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,kernel_initializer = RandomUniform(minval=weight_initialize_parameter1,maxval=weight_initialize_parameter2))(cov'+str(i+1)+'))')
                                    elif weight_initialize_method=='TruncatedNormal':
                                        exec('cov'+str(i+1)+'=ZeroPadding2D(padding=((math.floor(math.floor((cov'+str(i+1)+'.shape[1]-((cov'+str(i+1)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((cov'+str(i+1)+'.shape[1]-((cov'+str(i+1)+'.shape[1]-cov_kernelsize+1)/cov_strides)))/2)),(math.floor(math.floor((cov'+str(i+1)+'.shape[2]-((cov'+str(i+1)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2),math.ceil(math.floor((cov'+str(i+1)+'.shape[2]-((cov'+str(i+1)+'.shape[2]-cov_kernelsize+1)/cov_strides)))/2))))(LocallyConnected2D(baselayer,(cov_kernelsize,cov_kernelsize),strides=cov_strides,activation=activate,kernel_initializer = TruncatedNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2,metrics="MeanSquaredError"))(cov'+str(i+1)+'))')
                    if ifnormalization=='yes':
                        if normalization_method=='batchnormalization':
                            exec('norm'+str(i+1)+'=BatchNormalization(axis=-1)(cov'+str(i+1)+')')
                        elif normalization_method=='layernormalization':
                            exec('norm'+str(i+1)+'=LayerNormalization(axis=-1)(cov'+str(i+1)+')')
                k=k+1
                l=l-1
            if if_last_act!='no':
                if ifnormalization=='yes':
                    outputs=eval('Conv2D(vy.shape[3], (1, 1),strides=(1,1),padding="same",activation=if_last_act)(norm'+str(2*downnum-1)+')')
                else:
                    outputs=eval('Conv2D(vy.shape[3], (1, 1),strides=(1,1),padding="same",activation=if_last_act)(cov'+str(2*downnum-1)+')')
            else:
                if ifnormalization=='yes':
                    outputs=eval('Conv2D(vy.shape[3], (1, 1),strides=(1,1),padding="same")(norm'+str(2*downnum-1)+')')
                else:
                    outputs=eval('Conv2D(vy.shape[3], (1, 1),strides=(1,1),padding="same")(cov'+str(2*downnum-1)+')')
            model=eval('Model(inputs=inputs, outputs=outputs)')
            if optimizer == 'SGD':
                opt = SGD(lr = learning_rate)
            elif optimizer == 'Adam':
                opt = Adam(lr = learning_rate)
            model.compile(loss=loss,optimizer=opt,metrics=[metric])
        elif if_best_mode=='yes' or if_best_mode=='load':
            model=load_model(modelpath)
        trainx=np.nan_to_num(trainx,nan=0)
        testx=np.nan_to_num(testx,nan=0)
        trainy=np.nan_to_num(trainy,nan=0)
        testy=np.nan_to_num(testy,nan=0)
        if if_print_model=='yes':
            print(model.summary())
        if epochs!=0:
            H = model.fit(trainx,trainy,validation_data=(testx,testy),epochs = epochs,batch_size=batch_size)
        r=np.zeros((testx.shape[1],testx.shape[2],testy.shape[3]))
        p=np.zeros((testx.shape[1],testx.shape[2],testy.shape[3]))
        f1score=np.zeros((testx.shape[1],testx.shape[2],testy.shape[3]))
        accuracyscore=np.zeros((testx.shape[1],testx.shape[2],testy.shape[3]))
        recallscore=np.zeros((testx.shape[1],testx.shape[2],testy.shape[3]))
        precisionscore=np.zeros((testx.shape[1],testx.shape[2],testy.shape[3]))
        predicty = model.predict(testx,batch_size=batch_size)
        predicty=predicty.reshape(predicty.shape[0],predicty.shape[1],predicty.shape[2],predicty.shape[3])
        for i in range(testx.shape[1]):
            for j in range(testx.shape[2]):
                for k in range(testy.shape[3]):
                    if task_mode=='regression':
                        r[i,j,k],p[i,j,k] = pearsonr(predicty[:,i,j,k],testy[:,i,j,k])
                        r[i,j,k]=np.nan_to_num(r[i,j,k],nan=0)
                    elif task_mode=='binary_classify':
                        predicty[:,i,j,k]=[int(round(predicty[l,i,j,k],0)) for l in range(predicty.shape[0])]
                        if metrics=='Recall':
                            r[i,j,k]=recall_score(testy[:,i,j,k], predicty[:,i,j,k])
                        elif metrics=='Precision':
                            r[i,j,k]=precision_score(testy[:,i,j,k], predicty[:,i,j,k])
                        else:
                            r[i,j,k]=accuracy_score(testy[:,i,j,k], predicty[:,i,j,k])
                        p[i,j,k]=0
                        f1score[i,j,k]=f1_score(testy[:,i,j,k],predicty[:,i,j,k])
                        accuracyscore[i,j,k]=accuracy_score(testy[:,i,j,k],predicty[:,i,j,k])
                        recallscore[i,j,k]=recall_score(testy[:,i,j,k],predicty[:,i,j,k])
                        precisionscore=precision_score(testy[:,i,j,k],predicty[:,i,j,k])
                    elif task_mode=='multi_classify':
                        r[i,j,k]=accuracy_score(testy[:,i,j,k], predicty[:,i,j,k])
                        p[i,j,k]=0
        if ifmute == 'no':
            if task_mode=='regression':
                print('相关系数',np.nanmean(r))
            elif task_mode=='binary_classify':
                print('召回率+精确率',np.nanmean(f1score),'准确率',np.nanmean(accuracyscore),'召回率',np.nanmean(recallscore),'精确率',np.nanmean(precisionscore))
            elif task_mode=='multi_classify':
                print('准确率',np.nanmean(r))
        if ifweight=='yes':
            weights=np.zeros((testy.shape[3],testx.shape[3]))
            weight_more=np.zeros((testy.shape[3],testx.shape[3]))
            for i in range(testy.shape[3]):
                for j in range(testx.shape[3]):
                    testx_new=copy.deepcopy(testx)
                    weight=[]
                    for k in range(10):
                        per=np.random.permutation(testx.shape[0])
                        testx_shuffle=testx[per,:,:,j]
                        testx_new[:,:,:,j]=testx_shuffle
                        predicty_new=np.array(model.predict(testx_new,batch_size=batch_size))
                        for l in range(testy.shape[1]):
                            for m in range(testy.shape[2]):
                                if task_mode=='regression':
                                    weight.append(sklearn.metrics.mean_squared_error(testy[:,l,m,i],predicty_new[:,l,m,i])-sklearn.metrics.mean_squared_error(testy[:,l,m,i],predicty[:,l,m,i]))
                                else:
                                    weight.append(sklearn.metrics.log_loss(testy[:,l,m,i],predicty_new[:,l,m,i])-sklearn.metrics.log_loss(testy[:,l,m,i],predicty[:,l,m,i]))
                    weight_more[i,j]=np.nanmean(weight)
            for i in range(testy.shape[3]):
                for j in range(testx.shape[3]):
                    weights[i,j]=(weight_more[i,j]/np.nansum(weight_more[i,:]))*100
                    print('预报因子',j+1,'对预报值',i+1,'的贡献：',np.array(weights[i,j]),'％')
                print('\n')
        if ifsave=='yes':
            model.save(savepath)
    return model,predicty,testy,r,p,weights

In [9]:
model,predicty,testy,r,p,weights=Auto_Unet(CHM_PRE,vx_new,5,3,test_size=0.2,task_mode='regression',if_best_mode='no',modelpath=None,ifrandom_split='no',baselayer=16,cov_method='cov',cov_kernelsize=3,cov_strides=1,pool_strides=2,if_weight_initialize='no',weight_initialize_method='TruncatedNormal',weight_initialize_parameter1=0.00,weight_initialize_parameter2=0.05,activate='tanh',if_last_act='no',if_print_model='yes',loss_function='default',optimizer='Adam',metrics='default',learning_rate=0.001,epochs=500,batch_size=40,ifweight='no',ifmute='no',ifsave='yes',savepath=r'E:/xuanxuan/xuanxuan_unet_classify_regress',device='gpu')

Visible devices cannot be modified after being initialized


C:\Users\TBYC\AppData\Roaming\Python\Python39\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 143, 247, 7  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 143, 247, 16  1024        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 zero_padding2d (ZeroPadding2D)  (None, 143, 247, 16  0          ['conv2d[0][0]']                 
                                )                                                             

C:\Users\TBYC\AppData\Local\Temp\ipykernel_107444\143081705.py:851: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  r[i,j,k],p[i,j,k] = pearsonr(predicty[:,i,j,k],testy[:,i,j,k])


相关系数 0.2763944351391797


INFO:tensorflow:Assets written to: E:/xuanxuan/xuanxuan_unet_classify_regress\assets


INFO:tensorflow:Assets written to: E:/xuanxuan/xuanxuan_unet_classify_regress\assets
